In [1]:
#Importar paquetesgroup
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#Importamos variables temporales
now = datetime.now()
ayer = datetime.now() - timedelta(days=1)
anteayer = datetime.now() - timedelta(days=2)

# la fecha de corte es la más actual
fecha_corte = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y") # es la fecha actual en formato dd/mm/yyyy
print(now.strftime("%d") + '-' + now.strftime("%m") + '-' + now.strftime("%Y"))

23-04-2024


In [3]:
# Para limpiar variables duplicadas luego de un merge
def postmerge(df):
    replace_list = [i for i in df.columns if (i.endswith('_x'))]
    replace_list = [i[:-2] for i in replace_list]
    for i in replace_list:
        df[i] = df[i+'_x'].fillna(df[i+'_y'])
    df.drop([i for i in df.columns if (i.endswith('_x'))|(i.endswith('_y'))], inplace=True, axis=1)
    return df

# Para crear el COD_ID de la ejecutora
def crear_id(df, NIVEL_GOB, SECTOR, PLIEGO, EJECUTORA):
    df = df.assign(COD_ID = np.where(df[NIVEL_GOB].str.startswith('2'), (df[PLIEGO].str.split('. ', n=1).str[0] +"-" + df[EJECUTORA].str.split('. ', n=1).str[0]),
                                     (df[SECTOR].str.split('. ', n=1).str[0] +"-" + df[PLIEGO].str.split('. ', n=1).str[0] +"-"+ df[EJECUTORA].str.split('. ', n=1).str[0])))
    return df

In [4]:
print(datetime.now())

#Importar bases de datos
bd_gastos_tr = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'Data_Gastos_TipoRecurso_2018-2024_9Abr2024_2' + '.xlsx'), dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_ingresos_tr = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'Data_Ingresos_TipoRecurso_2018-2024_9Abr2024' + '.xlsx'), dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_sd = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'SD 20.04.2024 - GR GL' + '.xlsx'),sheet_name='Hoja1', dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_rubro18 = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'SB_Rubro18_06042024' + '.xlsx'), dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_rubro15 = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'SB_Foncor_06042024' + '.xlsx'), dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_R18= pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'CANON_190424_VF_sindecimal - GRGL' + '.xlsx'),  sheet_name='Hoja3', dtype={'ANO_EJE': int, 'SEC_EJEC': str})
bd_seg23 = pd.read_csv(os.path.join(r'Y:\Data_CSV\SIAFGasto2023', 'SeguimientoGasto_2023_04032024' + '.csv'))
bd_seg24 = pd.read_csv(os.path.join(r'Y:\Data_CSV\SIAFGasto2024', 'SeguimientoGasto_2024_'+ fecha_corte +'.csv'))
dict23 = pd.read_excel(os.path.join(r'Y:\Data_CSV\SIAFGasto2023', 'diccionarios_seguimiento_2023' + '.xlsx'), sheet_name='COD_CADENA_GASTO')
dict24 = pd.read_excel(os.path.join(r'Y:\Data_CSV\SIAFGasto2024', 'diccionarios_seguimiento_2024' + '.xlsx'), sheet_name='COD_CADENA_GASTO')
dict231 = pd.read_excel(os.path.join(r'Y:\Data_CSV\SIAFGasto2023', 'diccionarios_seguimiento_2023' + '.xlsx'), sheet_name='COD_ID')
dict241 = pd.read_excel(os.path.join(r'Y:\Data_CSV\SIAFGasto2024', 'diccionarios_seguimiento_2024' + '.xlsx'), sheet_name='COD_ID')
print(datetime.now())

2024-04-23 18:01:40.275181
2024-04-23 18:02:54.516144


# 1.Analisis Tipo Recurso PMG 2025 -2027

## Informaciòn Gastos

### Conversión de BD (a nivel de tipo de recurso) - 2018 - 2023

In [181]:
bd_gastos_tr.info()
bd_gastos_tr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107579 entries, 0 to 107578
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ANO_EJE           107579 non-null  int32  
 1   NIVEL_GOB         107579 non-null  object 
 2   SECTOR            107579 non-null  object 
 3   PLIEGO            107579 non-null  object 
 4   UE                107579 non-null  object 
 5   SEC_EJEC          107579 non-null  object 
 6   FUENTE            107579 non-null  object 
 7   RUBRO             107579 non-null  object 
 8   TIPO_RECURSO      107579 non-null  object 
 9   CATEGORIA_GTO     107579 non-null  object 
 10  TIPO_TRANSACCION  107579 non-null  object 
 11  GENERICA          107579 non-null  object 
 12  SUBGENERICA       107579 non-null  object 
 13  SUBGENERICA_DET   107579 non-null  object 
 14  ESPECIFICA        107579 non-null  object 
 15  ESPECIFICA_DET    107579 non-null  object 
 16  PIA               10

,ANO_EJE,NIVEL_GOB,SECTOR,PLIEGO,UE,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,...,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,CERTIFICACION,COMPROMISO_ANUAL,DEVENGADO
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,11. OTROS SERVICIOS,99. SERVICIOS DIVERSOS,0,0,0.0,0.0,211584.99
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,6. REPUESTOS Y ACCESORIOS,1. REPUESTOS Y ACCESORIOS,1. DE VEHICULOS,0,0,0.0,0.0,5100.00
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,7. SERVICIOS PROFESIONALES Y TECNICOS,"1. SERVICIOS DE CONSULTORIAS, ASESORIAS Y SIMI...",99. OTROS SERVICIOS SIMILARES,0,0,0.0,0.0,8460.00
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,99. COMPRA DE OTROS BIENES,1. COMPRA DE OTROS BIENES,1. HERRAMIENTAS,0,0,0.0,0.0,12773.00
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",19. SUB CUENTA - PARTICIPACIONES FED,5. GASTOS CORRIENTES,...,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,11. SUMINISTROS PARA MANTENIMIENTO Y REPARACION,1. SUMINISTROS PARA MANTENIMIENTO Y REPARACION,1. PARA EDIFICIOS Y ESTRUCTURAS,0,0,0.0,0.0,39520.50


In [182]:
hist_gastos = bd_gastos_tr.copy()
hist_gastos.rename(columns={'NIVEL_GOB' : 'NIVEL_GOBIERNO',
                            'CERTIFICACION' : 'CERTIFICADO',
                            'DEVENGADO' : 'TOTAL_DEVENGADO'}, inplace=True)

hist_gastos = hist_gastos[hist_gastos['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']  
hist_gastos['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"
hist_gastos= hist_gastos[hist_gastos['FUENTE'] == '5. RECURSOS DETERMINADOS']
hist_gastos = hist_gastos[((hist_gastos["RUBRO"].str[:2].isin(["15"]))|((hist_gastos["RUBRO"].str[:2].isin(["18"]))&(hist_gastos["TIPO_RECURSO"].str[:2].isin(["27"]))))]
hist_gastos['RUBRO'] = hist_gastos['RUBRO'].replace({'18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES':
                                                     '15. FONDO DE COMPENSACION REGIONAL - FONCOR'})
hist_gastos= hist_gastos[hist_gastos['ANO_EJE'].notna()]
hist_gastos['CATEGORIA_GTO'].replace({'5. GASTOS CORRIENTES': '5. GASTO CORRIENTE',
                                      '6. GASTOS DE CAPITAL': '6. GASTO DE CAPITAL'}, inplace=True)
hist_gastos1823 = hist_gastos[hist_gastos['ANO_EJE'] == 2023]
hist_gastos1823.info()
hist_gastos1823.head(98942)

<class 'pandas.core.frame.DataFrame'>
Index: 5181 entries, 78782 to 98877
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           5181 non-null   int32  
 1   NIVEL_GOBIERNO    5181 non-null   object 
 2   SECTOR            5181 non-null   object 
 3   PLIEGO            5181 non-null   object 
 4   UE                5181 non-null   object 
 5   SEC_EJEC          5181 non-null   object 
 6   FUENTE            5181 non-null   object 
 7   RUBRO             5181 non-null   object 
 8   TIPO_RECURSO      5181 non-null   object 
 9   CATEGORIA_GTO     5181 non-null   object 
 10  TIPO_TRANSACCION  5181 non-null   object 
 11  GENERICA          5181 non-null   object 
 12  SUBGENERICA       5181 non-null   object 
 13  SUBGENERICA_DET   5181 non-null   object 
 14  ESPECIFICA        5181 non-null   object 
 15  ESPECIFICA_DET    5181 non-null   object 
 16  PIA               5181 non-null   int64  


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,UE,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,...,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,CERTIFICADO,COMPROMISO_ANUAL,TOTAL_DEVENGADO
78782,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,"3. ADQUISICION DE VEHICULOS, MAQUINARIAS Y OTROS","2. ADQUISICION DE MAQUINARIAS, EQUIPO Y MOBILI...",2. PARA INSTALACIONES EDUCATIVAS,1. MAQUINAS Y EQUIPOS,0,0,0.0,0.0,2500.00
78783,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,"3. ADQUISICION DE VEHICULOS, MAQUINARIAS Y OTROS","2. ADQUISICION DE MAQUINARIAS, EQUIPO Y MOBILI...",9. ADQUISICION DE MAQUINARIA Y EQUIPO DIVERSOS,5. EQUIPOS E INSTRUMENTOS DE MEDICION,0,0,0.0,0.0,6516.00
78784,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,8. OTROS GASTOS DE ACTIVOS NO FINANCIEROS,1. OTROS GASTOS DE ACTIVOS NO FINANCIEROS,4. OTROS GASTOS DIVERSOS DE ACTIVOS NO FINANCI...,2. GASTO POR LA COMPRA DE BIENES,0,0,0.0,0.0,620685.30
78785,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,2. CONSTRUCCION DE EDIFICIOS Y ESTRUCTURAS,3. OTRAS ESTRUCTURAS,5. AGUA Y SANEAMIENTO,5. COSTO DE CONSTRUCCION POR ADMINISTRACION DI...,0,0,0.0,0.0,1290804.70
78786,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,2. CONSTRUCCION DE EDIFICIOS Y ESTRUCTURAS,2. EDIFICIOS O UNIDADES NO RESIDENCIALES,3. INSTALACIONES MEDICAS,5. COSTO DE CONSTRUCCION POR ADMINISTRACION DI...,0,0,0.0,0.0,3198174.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98476,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,100. AGRICULTURA LIMA,1228,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,2. CONSTRUCCION DE EDIFICIOS Y ESTRUCTURAS,3. OTRAS ESTRUCTURAS,4. INFRAESTRUCTURA AGRICOLA,5. COSTO DE CONSTRUCCION POR ADMINISTRACION DI...,0,0,0.0,0.0,5070851.36
98477,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,100. AGRICULTURA LIMA,1228,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,8. OTROS GASTOS DE ACTIVOS NO FINANCIEROS,1. OTROS GASTOS DE ACTIVOS NO FINANCIEROS,4. OTROS GASTOS DIVERSOS DE ACTIVOS NO FINANCI...,4. GASTO POR LAUDOS ARBITRALES O SENTENCIAS VI...,0,11220,11220.0,11220.0,0.00
98480,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,100. AGRICULTURA LIMA,1228,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FI

### Informaciòn 2024 (para PMG 2025 -2027)

In [183]:
gastos_tr24 = hist_gastos.copy()
gastos_tr24 = gastos_tr24[gastos_tr24['ANO_EJE'] == 2024]
gastos_tr24= gastos_tr24[gastos_tr24['ANO_EJE'].notna()]
gastos_tr24 = gastos_tr24.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO', 'CATEGORIA_GTO', 'GENERICA'])['TOTAL_DEVENGADO'].sum().reset_index()
gastos_tr24.info()
gastos_tr24.head(359)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ANO_EJE          110 non-null    int32  
 1   NIVEL_GOBIERNO   110 non-null    object 
 2   SECTOR           110 non-null    object 
 3   PLIEGO           110 non-null    object 
 4   FUENTE           110 non-null    object 
 5   RUBRO            110 non-null    object 
 6   TIPO_RECURSO     110 non-null    object 
 7   CATEGORIA_GTO    110 non-null    object 
 8   GENERICA         110 non-null    object 
 9   TOTAL_DEVENGADO  110 non-null    float64
dtypes: float64(1), int32(1), object(8)
memory usage: 8.3+ KB


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,GENERICA,TOTAL_DEVENGADO
0,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. 0,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,26639330.24
1,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.00
2,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.00
3,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. 0,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,3130.00
4,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. 0,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,26227224.46
...,...,...,...,...,...,...,...,...,...,...
105,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.00
106,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. 0,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,97370.73
107,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. 0,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,15456776.58
108,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.00


In [184]:
#Información historica
gastos_tr_rd_gr = pd.concat([hist_gastos1823, gastos_tr24], ignore_index=True)
gastos_tr_rd_gr.info()
gastos_tr_rd_gr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           5291 non-null   int32  
 1   NIVEL_GOBIERNO    5291 non-null   object 
 2   SECTOR            5291 non-null   object 
 3   PLIEGO            5291 non-null   object 
 4   UE                5181 non-null   object 
 5   SEC_EJEC          5181 non-null   object 
 6   FUENTE            5291 non-null   object 
 7   RUBRO             5291 non-null   object 
 8   TIPO_RECURSO      5291 non-null   object 
 9   CATEGORIA_GTO     5291 non-null   object 
 10  TIPO_TRANSACCION  5181 non-null   object 
 11  GENERICA          5291 non-null   object 
 12  SUBGENERICA       5181 non-null   object 
 13  SUBGENERICA_DET   5181 non-null   object 
 14  ESPECIFICA        5181 non-null   object 
 15  ESPECIFICA_DET    5181 non-null   object 
 16  PIA               5181 non-null   float64


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,UE,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,CATEGORIA_GTO,...,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,CERTIFICADO,COMPROMISO_ANUAL,TOTAL_DEVENGADO
0,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,"3. ADQUISICION DE VEHICULOS, MAQUINARIAS Y OTROS","2. ADQUISICION DE MAQUINARIAS, EQUIPO Y MOBILI...",2. PARA INSTALACIONES EDUCATIVAS,1. MAQUINAS Y EQUIPOS,0.0,0.0,0.0,0.0,2500.00
1,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,"3. ADQUISICION DE VEHICULOS, MAQUINARIAS Y OTROS","2. ADQUISICION DE MAQUINARIAS, EQUIPO Y MOBILI...",9. ADQUISICION DE MAQUINARIA Y EQUIPO DIVERSOS,5. EQUIPOS E INSTRUMENTOS DE MEDICION,0.0,0.0,0.0,0.0,6516.00
2,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,8. OTROS GASTOS DE ACTIVOS NO FINANCIEROS,1. OTROS GASTOS DE ACTIVOS NO FINANCIEROS,4. OTROS GASTOS DIVERSOS DE ACTIVOS NO FINANCI...,2. GASTO POR LA COMPRA DE BIENES,0.0,0.0,0.0,0.0,620685.30
3,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,2. CONSTRUCCION DE EDIFICIOS Y ESTRUCTURAS,3. OTRAS ESTRUCTURAS,5. AGUA Y SANEAMIENTO,5. COSTO DE CONSTRUCCION POR ADMINISTRACION DI...,0.0,0.0,0.0,0.0,1290804.70
4,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,6. GASTO DE CAPITAL,...,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,2. CONSTRUCCION DE EDIFICIOS Y ESTRUCTURAS,2. EDIFICIOS O UNIDADES NO RESIDENCIALES,3. INSTALACIONES MEDICAS,5. COSTO DE CONSTRUCCION POR ADMINISTRACION DI...,0.0,0.0,0.0,0.0,3198174.74


### Cálculo de ratio por GG y Tipo de Recurso

In [185]:
ratios_preliminar = gastos_tr_rd_gr.copy()
ratios_preliminar = ratios_preliminar.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO', 'CATEGORIA_GTO', 'GENERICA'])['TOTAL_DEVENGADO'].sum().reset_index()
ratios_preliminar['CL_TIPO_RECURSO'] = ' '
ratios_preliminar['CL_TIPO_RECURSO'] = 'FONCOR'                                              

ratios_preliminar = ratios_preliminar.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO', 'CATEGORIA_GTO', 'GENERICA'])['TOTAL_DEVENGADO'].sum().reset_index()
ratios_preliminar['DEN_DEVENGADO'] = ratios_preliminar.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'])['TOTAL_DEVENGADO'].transform('sum').astype(float)

#Generar ratio_
ratios_preliminar['ratio_'] = ratios_preliminar['TOTAL_DEVENGADO'] / ratios_preliminar['DEN_DEVENGADO']

# Filtrando los datos para mantener solo los años entre 2018 y 2024
ratios_preliminar = ratios_preliminar[(ratios_preliminar['ANO_EJE'] >= 2018) & (ratios_preliminar['ANO_EJE'] <= 2024)]

# Eliminando filas donde ratio_ sea nulo

ratios_preliminar = ratios_preliminar.dropna(subset=['ratio_'])
# Definir las estadísticas a calcular
estadisticas = ['min', 'mean', 'max']

# Iterar sobre cada estadística
for estadistica in estadisticas:
    # Calcular la estadística 'estadistica' del ratio_ por grupos y agregarla al dataframe
    ratios_preliminar[f'{estadistica}_ratio_'] = ratios_preliminar.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA', 'CL_TIPO_RECURSO'])['ratio_'].transform(estadistica)

# Ordenar el dataframe por las columnas especificadas
ratios_preliminar.sort_values(by=['PLIEGO', 'GENERICA', 'CL_TIPO_RECURSO'], inplace=True)

# Eliminar filas duplicadas en el dataframe
ratios_preliminar.drop_duplicates(inplace=True)

# Reshape wide
ratios_preliminar = ratios_preliminar.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'], columns='CL_TIPO_RECURSO', values=['min_ratio_', 'mean_ratio_', 'max_ratio_']).reset_index().rename_axis(None)
ratios_preliminar.columns = [f'{i}{j}' for i, j in ratios_preliminar.columns]
ratios_preliminar.info()
ratios_preliminar.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NIVEL_GOBIERNO     69 non-null     object 
 1   SECTOR             69 non-null     object 
 2   PLIEGO             69 non-null     object 
 3   FUENTE             69 non-null     object 
 4   RUBRO              69 non-null     object 
 5   CATEGORIA_GTO      69 non-null     object 
 6   GENERICA           69 non-null     object 
 7   max_ratio_FONCOR   69 non-null     float64
 8   mean_ratio_FONCOR  69 non-null     float64
 9   min_ratio_FONCOR   69 non-null     float64
dtypes: float64(3), object(7)
memory usage: 5.5+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,max_ratio_FONCOR,mean_ratio_FONCOR,min_ratio_FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.006477,0.003239,0.000000
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.000000,0.000000,0.000000
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,0.996761,0.993523
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.000119,0.000119,0.000119
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,0.999940,0.999881


In [186]:
gastos = gastos_tr_rd_gr.copy()
gastos = gastos.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO'])['TOTAL_DEVENGADO'].sum().reset_index()
gastos['CL_TIPO_RECURSO'] = ' '
gastos['CL_TIPO_RECURSO'] = 'FONCOR' 

gastos = gastos.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'])['TOTAL_DEVENGADO'].sum().reset_index()
gastos.info()
gastos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ANO_EJE          49 non-null     int32  
 1   NIVEL_GOBIERNO   49 non-null     object 
 2   SECTOR           49 non-null     object 
 3   PLIEGO           49 non-null     object 
 4   FUENTE           49 non-null     object 
 5   RUBRO            49 non-null     object 
 6   CL_TIPO_RECURSO  49 non-null     object 
 7   TOTAL_DEVENGADO  49 non-null     float64
dtypes: float64(1), int32(1), object(6)
memory usage: 3.0+ KB


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CL_TIPO_RECURSO,TOTAL_DEVENGADO
0,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,1.917627e+08
1,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,6.510635e+07
2,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,8.511376e+07
3,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,2.651948e+08
4,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,2.037461e+08


## Informaciòn Ingresos

### Conversión de BD (a nivel de tipo de recurso) 2018 - 2023

In [187]:
bd_ingresos_tr.info()
bd_ingresos_tr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25366 entries, 0 to 25365
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           25366 non-null  int32  
 1   NIVEL_GOB         25366 non-null  object 
 2   SECTOR            25366 non-null  object 
 3   PLIEGO            25366 non-null  object 
 4   EJECUTORA         25366 non-null  object 
 5   SEC_EJEC          25366 non-null  object 
 6   FUENTE            25366 non-null  object 
 7   RUBRO             25366 non-null  object 
 8   TIPO_RECURSO      25366 non-null  object 
 9   TIPO_TRANSACCION  25366 non-null  object 
 10  GENERICA          25366 non-null  object 
 11  SUBGENERICA       25366 non-null  object 
 12  SUBGENERICA_DET   25366 non-null  object 
 13  ESPECIFICA        25366 non-null  object 
 14  ESPECIFICA_DET    25366 non-null  object 
 15  PIA               25366 non-null  int64  
 16  PIM               25366 non-null  int64 

,ANO_EJE,NIVEL_GOB,SECTOR,PLIEGO,EJECUTORA,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,TIPO_TRANSACCION,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,RECAUDADO
0,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",99. SIN TIPO DE RECURSO,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,2. DONACIONES DE CAPITAL,3. DE OTRAS UNIDADES DE GOBIERNO,1. DE OTRAS UNIDADES DE GOBIERNO,5. DE FONDOS PUBLICOS,0,36400,0.00
1,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0,0,94377.00
2,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",D. SUB CUENTA - PARTICIPACIONES - DNTP,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,2484.26
3,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",H. SUB CUENTA - CANON MINERO,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,1546.60
4,2018,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",L. SUB CUENTA - CANON FORESTAL,1. INGRESOS PRESUPUESTARIOS,9. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,1. SALDOS DE BALANCE,0,0,69121.51


In [188]:
hist_ingresos = bd_ingresos_tr.copy()
# Reemplazar comas por puntos en la columna 'recaudacion'
hist_ingresos.rename(columns={'RECAUDADO': 'RECAUDACION',
                              'NIVEL_GOB': 'NIVEL_GOBIERNO'}, inplace=True)

# Convertir la columna 'recaudacion' a tipo numérico si es posible
hist_ingresos['RECAUDACION'] = hist_ingresos['RECAUDACION'].astype(float)

# Seleccionar solo GOBIERNOS REGIONALES
hist_ingresos= hist_ingresos[hist_ingresos['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']
# Seleccionar solo RUBRO 18 - CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES.
hist_ingresos= hist_ingresos[hist_ingresos['FUENTE'] == '5. RECURSOS DETERMINADOS']
hist_ingresos['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"

hist_ingresos = hist_ingresos[((hist_ingresos["RUBRO"].str[:2].isin(["15"]))|((hist_ingresos["RUBRO"].str[:2].isin(["18"]))&(hist_ingresos["TIPO_RECURSO"].str[:2].isin(["27"]))))]
hist_ingresos['RUBRO'] = hist_ingresos['RUBRO'].replace({'18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADUANAS Y PARTICIPACIONES':
                                                     '15. FONDO DE COMPENSACION REGIONAL - FONCOR'})

# Dropear el año 2024, porque esa informaciòn la brinda DPSP
hist_ingresos1823 = hist_ingresos[hist_ingresos['ANO_EJE'] != 2024]

hist_ingresos1823.info()
hist_ingresos1823.head(1737)

<class 'pandas.core.frame.DataFrame'>
Index: 1737 entries, 9690 to 22323
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           1737 non-null   int32  
 1   NIVEL_GOBIERNO    1737 non-null   object 
 2   SECTOR            1737 non-null   object 
 3   PLIEGO            1737 non-null   object 
 4   EJECUTORA         1737 non-null   object 
 5   SEC_EJEC          1737 non-null   object 
 6   FUENTE            1737 non-null   object 
 7   RUBRO             1737 non-null   object 
 8   TIPO_RECURSO      1737 non-null   object 
 9   TIPO_TRANSACCION  1737 non-null   object 
 10  GENERICA          1737 non-null   object 
 11  SUBGENERICA       1737 non-null   object 
 12  SUBGENERICA_DET   1737 non-null   object 
 13  ESPECIFICA        1737 non-null   object 
 14  ESPECIFICA_DET    1737 non-null   object 
 15  PIA               1737 non-null   int64  
 16  PIM               1737 non-null   int64  
 

,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,TIPO_TRANSACCION,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,RECAUDACION
9690,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,26078.27
9692,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,1. DONACIONES Y TRANSFERENCIAS CORRIENTES,4. POR PARTICIPACIONES DE RECURSOS DETERMINADOS,7. FONDO DE COMPENSACION REGIONAL,1. FONDO DE COMPENSACION REGIONAL,0,0,23445032.00
9708,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,002. GERENCIA SUB REGIONAL BAGUA,953,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,5860.10
9710,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,002. GERENCIA SUB REGIONAL BAGUA,953,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,1. DONACIONES Y TRANSFERENCIAS CORRIENTES,4. POR PARTICIPACIONES DE RECURSOS DETERMINADOS,7. FONDO DE COMPENSACION REGIONAL,1. FONDO DE COMPENSACION REGIONAL,0,0,6394773.00
9717,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,003. GERENCIA SUB REGIONAL CONDORCANQUI,1023,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,4185.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22153,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,100. AGRICULTURA LIMA,1228,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,469785.81
22161,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,100. AGRICULTURA LIMA,1228,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,1. DONACIONES Y TRANSFERENCIAS CORRIENTES,4. POR PARTICIPACIONES DE RECURSOS DETERMINADOS,7. FONDO DE COMPENSACION REGIONAL,1. FONDO DE COMPENSACION REGIONAL,86437737,86437737,0.00
22305,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0,0,495768.55
22308,2023,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,001. GOBIERNO REGIONAL CALLAO,1028,5. RECURSOS D

### Informaciòn 2024 (para PMG 2025-2027)

In [189]:
ingresos_tr24 = hist_ingresos.copy()
ingresos_tr24 = ingresos_tr24[ingresos_tr24['ANO_EJE'] == 2024]
ingresos_tr24 = ingresos_tr24.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO'])['RECAUDACION'].sum().reset_index()
ingresos_tr24.info()
ingresos_tr24.head(337)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ANO_EJE         49 non-null     int32  
 1   NIVEL_GOBIERNO  49 non-null     object 
 2   SECTOR          49 non-null     object 
 3   PLIEGO          49 non-null     object 
 4   FUENTE          49 non-null     object 
 5   RUBRO           49 non-null     object 
 6   TIPO_RECURSO    49 non-null     object 
 7   RECAUDACION     49 non-null     float64
dtypes: float64(1), int32(1), object(6)
memory usage: 3.0+ KB


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_RECURSO,RECAUDACION
0,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,3.726268e+07
1,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,0.000000e+00
2,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1.631972e+08
3,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,0.000000e+00
4,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,2.046762e+07
5,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,0.000000e+00
6,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,2.121578e+08
7,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,0.000000e+00
8,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,5.142739e+07
9,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,0.000000e+00


In [190]:
#Información historica
ingresos_tr_rd_gr = pd.concat([hist_ingresos1823, ingresos_tr24], ignore_index=True)
ingresos_tr_rd_gr.info()
ingresos_tr_rd_gr.head(21942)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1786 entries, 0 to 1785
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO_EJE           1786 non-null   int32  
 1   NIVEL_GOBIERNO    1786 non-null   object 
 2   SECTOR            1786 non-null   object 
 3   PLIEGO            1786 non-null   object 
 4   EJECUTORA         1737 non-null   object 
 5   SEC_EJEC          1737 non-null   object 
 6   FUENTE            1786 non-null   object 
 7   RUBRO             1786 non-null   object 
 8   TIPO_RECURSO      1786 non-null   object 
 9   TIPO_TRANSACCION  1737 non-null   object 
 10  GENERICA          1737 non-null   object 
 11  SUBGENERICA       1737 non-null   object 
 12  SUBGENERICA_DET   1737 non-null   object 
 13  ESPECIFICA        1737 non-null   object 
 14  ESPECIFICA_DET    1737 non-null   object 
 15  PIA               1737 non-null   float64
 16  PIM               1737 non-null   float64


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,SEC_EJEC,FUENTE,RUBRO,TIPO_RECURSO,TIPO_TRANSACCION,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PIM,RECAUDACION
0,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0.0,0.0,2.607827e+04
1,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,1. DONACIONES Y TRANSFERENCIAS CORRIENTES,4. POR PARTICIPACIONES DE RECURSOS DETERMINADOS,7. FONDO DE COMPENSACION REGIONAL,1. FONDO DE COMPENSACION REGIONAL,0.0,0.0,2.344503e+07
2,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,002. GERENCIA SUB REGIONAL BAGUA,953,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0.0,0.0,5.860100e+03
3,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,002. GERENCIA SUB REGIONAL BAGUA,953,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,4. DONACIONES Y TRANSFERENCIAS,1. DONACIONES Y TRANSFERENCIAS CORRIENTES,4. POR PARTICIPACIONES DE RECURSOS DETERMINADOS,7. FONDO DE COMPENSACION REGIONAL,1. FONDO DE COMPENSACION REGIONAL,0.0,0.0,6.394773e+06
4,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,003. GERENCIA SUB REGIONAL CONDORCANQUI,1023,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,1. INGRESOS PRESUPUESTARIOS,5. OTROS INGRESOS,1. RENTAS DE LA PROPIEDAD,1. RENTAS DE LA PROPIEDAD FINANCIERA,1. INTERESES,1. INTERESES POR DEPOSITOS DISTINTOS DE RECURS...,0.0,0.0,4.185610e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,NaN,NaN,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
1782,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,NaN,NaN,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.953599e+08
1783,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,NaN,NaN,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,99. SIN TIPO DE RECURSO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
1784,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,NaN,NaN,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,27. SUB CUENTA - FONCOR - LEY 31069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.089403e+07


### Cálculo de saldo de balance por tipo de recurso 

In [191]:
ingresos = ingresos_tr_rd_gr.copy()
ingresos = ingresos.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_RECURSO'])['RECAUDACION'].sum().reset_index()
ingresos['CL_TIPO_RECURSO'] = ' '
ingresos['CL_TIPO_RECURSO'] = 'FONCOR'
ingresos = ingresos.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'])['RECAUDACION'].sum().reset_index()

ingresos.info()
ingresos.head(597)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ANO_EJE          97 non-null     int32  
 1   NIVEL_GOBIERNO   97 non-null     object 
 2   SECTOR           97 non-null     object 
 3   PLIEGO           97 non-null     object 
 4   FUENTE           97 non-null     object 
 5   RUBRO            97 non-null     object 
 6   CL_TIPO_RECURSO  97 non-null     object 
 7   RECAUDACION      97 non-null     float64
dtypes: float64(1), int32(1), object(6)
memory usage: 5.8+ KB


,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CL_TIPO_RECURSO,RECAUDACION
0,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,5.764276e+07
1,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,8.698624e+06
2,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,4.404231e+07
3,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,1.923907e+07
4,2021,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,4.489843e+07
...,...,...,...,...,...,...,...,...
92,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,460. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TACNA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,8.157221e+06
93,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,461. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TUMBES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,4.279348e+07
94,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,6.525916e+07
95,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,FONCOR,1.953599e+08


In [192]:
sb_ratios = pd.merge(gastos, ingresos, on=['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'], how='outer', validate = '1:1')
postmerge(sb_ratios)
sb_ratios = sb_ratios[(sb_ratios['ANO_EJE'] >= 2018) & (sb_ratios['ANO_EJE'] <= 2024)]
# Eliminar las filas donde 'recaudacion' sea igual a NaN y 'total_devengado' sea igual a 0
sb_ratios = sb_ratios.drop(sb_ratios[(sb_ratios['RECAUDACION'].isna()) & (sb_ratios['TOTAL_DEVENGADO'] == 0)].index)
# Eliminar las filas donde 'recaudacion' sea igual a 0 y 'total_devengado' sea igual a 0
sb_ratios = sb_ratios.drop(sb_ratios[(sb_ratios['RECAUDACION'] == 0) & (sb_ratios['TOTAL_DEVENGADO'] == 0)].index)
# Eliminar las filas donde 'recaudacion' sea igual a 0 y 'total_devengado' sea igual a NaN
sb_ratios = sb_ratios.drop(sb_ratios[(sb_ratios['RECAUDACION'] == 0) & (sb_ratios['TOTAL_DEVENGADO'].isna())].index)
# Eliminar las filas donde 'recaudacion' sea igual a NaN y 'total_devengado' sea igual a NaN
sb_ratios = sb_ratios.drop(sb_ratios[(sb_ratios['RECAUDACION'].isna()) & (sb_ratios['TOTAL_DEVENGADO'].isna())].index)

# Reemplazar los valores nulos en 'total_devengado' y 'recaudacion' con 0
sb_ratios['TOTAL_DEVENGADO'].fillna(0, inplace=True)
sb_ratios['RECAUDACION'].fillna(0, inplace=True)

# Calcular la variable 'sb' y reemplazar los valores negativos con 0
sb_ratios['sb'] = sb_ratios['RECAUDACION'] - sb_ratios['TOTAL_DEVENGADO']
sb_ratios['sb'] = np.where((sb_ratios['sb'] < 0), 0, sb_ratios['sb'])

# Calcular la suma acumulada de 'sb' por grupos y guardarla en 'den_sb'
sb_ratios['den_sb'] = sb_ratios.groupby(['ANO_EJE', 'NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['sb'].transform('sum')
sb_ratios['ratio_sb'] = sb_ratios['sb'] / sb_ratios['den_sb']

# Definir las estadísticas a calcular
estadisticas = ['min', 'mean', 'max']

# Iterar sobre cada estadística
for estadistica in estadisticas:
    # Calcular la estadística 'estadistica' del ratio_ por grupos y agregarla al dataframe
    sb_ratios[f'{estadistica}_ratio_sb_'] = sb_ratios.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CL_TIPO_RECURSO'])['ratio_sb'].transform(estadistica)

# Eliminar filas duplicadas en el dataframe
sb_ratios.drop_duplicates(inplace=True)

# Reshape wide
sb_ratios = sb_ratios.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], columns='CL_TIPO_RECURSO', values=['min_ratio_sb_', 'mean_ratio_sb_', 'max_ratio_sb_']).reset_index().rename_axis(None)
sb_ratios.columns = [f'{i}{j}' for i, j in sb_ratios.columns]

sb_ratios.info()
sb_ratios.head(27)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   NIVEL_GOBIERNO        25 non-null     object 
 1   SECTOR                25 non-null     object 
 2   PLIEGO                25 non-null     object 
 3   FUENTE                25 non-null     object 
 4   RUBRO                 25 non-null     object 
 5   max_ratio_sb_FONCOR   25 non-null     float64
 6   mean_ratio_sb_FONCOR  25 non-null     float64
 7   min_ratio_sb_FONCOR   25 non-null     float64
dtypes: float64(3), object(5)
memory usage: 1.7+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,max_ratio_sb_FONCOR,mean_ratio_sb_FONCOR,min_ratio_sb_FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0,1.0,1.0


# 2. Insumos PMG 2025-2027

## Anàlisis a nivel de gastos: ratio por generica de gasto y tipo de recurso

In [193]:
ratios_preliminar.info()
ratios_preliminar.head(139)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NIVEL_GOBIERNO     69 non-null     object 
 1   SECTOR             69 non-null     object 
 2   PLIEGO             69 non-null     object 
 3   FUENTE             69 non-null     object 
 4   RUBRO              69 non-null     object 
 5   CATEGORIA_GTO      69 non-null     object 
 6   GENERICA           69 non-null     object 
 7   max_ratio_FONCOR   69 non-null     float64
 8   mean_ratio_FONCOR  69 non-null     float64
 9   min_ratio_FONCOR   69 non-null     float64
dtypes: float64(3), object(7)
memory usage: 5.5+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,max_ratio_FONCOR,mean_ratio_FONCOR,min_ratio_FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.006477,0.003239,0.000000
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.000000,0.000000,0.000000
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,0.996761,0.993523
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.000119,0.000119,0.000119
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,1.000000,0.999940,0.999881
...,...,...,...,...,...,...,...,...,...,...
64,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.008878,0.004439,0.000000
65,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.031827,0.027558,0.023289
66,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.976711,0.966872,0.957033
67,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.006260,0.006260,0.006260


In [194]:
ratios_preliminar_red = ratios_preliminar.copy()
ratios_preliminar_red = ratios_preliminar_red[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA', 'mean_ratio_FONCOR']]
# Calcular el ratio tot_rat
ratios_preliminar_red['tot_rat'] = ratios_preliminar_red[['mean_ratio_FONCOR']].sum(axis=1)

#eliminar si tot_rat es =0 o . :
ratios_preliminar_red['tot_rat'] = round(ratios_preliminar_red['tot_rat'], 8)
ratios_preliminar_red = ratios_preliminar_red[(ratios_preliminar_red['tot_rat'].fillna(0) != 0)]

#Asumimos que ningún pliego debe financiar personal con RD a excepcion de GR Callao
ratios_preliminar_red = ratios_preliminar_red[~(((ratios_preliminar_red['GENERICA'].str[0] == '1') | (ratios_preliminar_red['GENERICA'].str[0] == '2')) & (ratios_preliminar_red['PLIEGO'].str[:3] != '464'))]
#Estamos cubriendo GG 2.8 en función al monto de Tesoro y no al histórico
ratios_preliminar_red = ratios_preliminar_red[(ratios_preliminar_red['GENERICA'].str[0] != '8')]
#Mantener las siguientes variables
ratios_preliminar_red = ratios_preliminar_red[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA', 'mean_ratio_FONCOR']]

#CANON Y SOBRECANON
#Corregir la participación de acuerdo a la participación de cada Genérica
# Calcular la suma acumulada de mean_ratio_C_SC_RM para cada grupo
ratios_preliminar_red['Tot1'] = ratios_preliminar_red.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['mean_ratio_FONCOR'].transform('sum')

# Calcular el ratio_f_c
ratios_preliminar_red['ratio_f_c'] = ratios_preliminar_red['mean_ratio_FONCOR'] / ratios_preliminar_red['Tot1']

# Corregir ratio_f_c si ratio_f es mayor que 0.2 y la primera letra de generica es "3"
ratios_preliminar_red['ratio_f_c'] = np.where((ratios_preliminar_red['ratio_f_c'] > 0.2) & (ratios_preliminar_red['GENERICA'].str[0] == '3'), 0.2, ratios_preliminar_red['ratio_f_c'])

# Calcular la suma acumulada de ratio_f_c para cada grupo
ratios_preliminar_red['Tot2'] = ratios_preliminar_red.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['ratio_f_c'].transform('sum')

# Calcular comp
ratios_preliminar_red['comp'] = 1 - ratios_preliminar_red['Tot2']

# Ajustar ratio_f_c si la primera letra de generica es "6"
ratios_preliminar_red['ratio_f_c'] = ratios_preliminar_red['ratio_f_c'].fillna(0)
ratios_preliminar_red['comp'] = ratios_preliminar_red['comp'].fillna(0)
ratios_preliminar_red['ratio_f_c'] = np.where(ratios_preliminar_red['GENERICA'].str[0] == '6', ratios_preliminar_red[['ratio_f_c', 'comp']].sum(axis=1), ratios_preliminar_red['ratio_f_c'])

# Eliminar columnas temporales
ratios_preliminar_red.drop(columns=['Tot1', 'Tot2'], inplace=True)

#RENTA DE ADUANA
#Corregir la participación de acuerdo a la participación de cada Genérica
# Calcular la suma acumulada de Tot_ra para cada grupo
#ratios_preliminar_red['Tot_ra'] = ratios_preliminar_red.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['mean_ratio_R_Aduana'].transform('sum')
# Calcular ratioratio_f_ra
#ratios_preliminar_red['ratio_f_ra'] = ratios_preliminar_red['mean_ratio_R_Aduana'] / ratios_preliminar_red['Tot_ra']

# Calcular la suma acumulada de ratio_f_ra para cada grupo
#ratios_preliminar_red['Totra2'] = ratios_preliminar_red.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'])['ratio_f_ra'].transform('sum')
#ratios_preliminar_red['compra'] = 1 - ratios_preliminar_red['Totra2']

# Ajustar ratio_f_ra si la primera letra de generica es "6"
#ratios_preliminar_red['ratio_f_ra']= ratios_preliminar_red['ratio_f_ra'].fillna(0)
#ratios_preliminar_red['compra']= ratios_preliminar_red['compra'].fillna(0)
#ratios_preliminar_red['ratio_f_ra'] = np.where(ratios_preliminar_red['GENERICA'].str[0] == '6', ratios_preliminar_red[['ratio_f_ra', 'compra']].sum(axis=1), ratios_preliminar_red['ratio_f_ra'])

# Eliminar columnas temporales
#ratios_preliminar_red.drop(columns=['Tot_ra', 'Totra2'], inplace=True)

# Eliminar filas duplicadas en el dataframe
sb_ratios.drop_duplicates(inplace=True)


ratios_preliminar_red.info()
ratios_preliminar_red.head(139)

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 68
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NIVEL_GOBIERNO     59 non-null     object 
 1   SECTOR             59 non-null     object 
 2   PLIEGO             59 non-null     object 
 3   FUENTE             59 non-null     object 
 4   RUBRO              59 non-null     object 
 5   CATEGORIA_GTO      59 non-null     object 
 6   GENERICA           59 non-null     object 
 7   mean_ratio_FONCOR  59 non-null     float64
 8   ratio_f_c          59 non-null     float64
 9   comp               59 non-null     float64
dtypes: float64(3), object(7)
memory usage: 5.1+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,mean_ratio_FONCOR,ratio_f_c,comp
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.003239,0.003239,0.000000e+00
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.996761,0.996761,0.000000e+00
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.000119,0.000119,0.000000e+00
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.999940,0.999881,0.000000e+00
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.001746,0.001745,0.000000e+00
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.000899,0.000899,0.000000e+00
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.997804,0.997356,0.000000e+00
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.051330,0.051363,0.000000e+00
10,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.090239,0.090296,0.000000e+00
11,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.857792,0.858341,0.000000e+00


## Adaptar a los ratios que nos interesan (Tipo de recurso de ingresos- SB)

In [195]:
sb_ratios_red = sb_ratios.copy()
sb_ratios_red = sb_ratios_red[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'mean_ratio_sb_FONCOR']]
# Calcular el ratio tot_rat
sb_ratios_red['tot_rat'] = sb_ratios_red[['mean_ratio_sb_FONCOR']].sum(axis=1)

# Calcular los ratios
sb_ratios_red['msb_c'] = sb_ratios_red['mean_ratio_sb_FONCOR'] / sb_ratios_red['tot_rat']
#sb_ratios_red['ratio_sb_fed'] = sb_ratios_red['mean_ratio_sb_FED'] / sb_ratios_red['tot_rat']
#sb_ratios_red['msb_fide'] = sb_ratios_red['mean_ratio_sb_FIDEICOMISO'] / sb_ratios_red['tot_rat']
#sb_ratios_red['msb_ot'] = sb_ratios_red['mean_ratio_sb_OTROS'] / sb_ratios_red['tot_rat']
#sb_ratios_red['ratio_sb_ra'] = sb_ratios_red['mean_ratio_sb_R_Aduana'] / sb_ratios_red['tot_rat']

# Eliminar columnas temporales
sb_ratios_red.drop(columns=['tot_rat', 'mean_ratio_sb_FONCOR'], inplace=True)

sb_ratios_red.info()
sb_ratios_red.head(26)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOBIERNO  25 non-null     object 
 1   SECTOR          25 non-null     object 
 2   PLIEGO          25 non-null     object 
 3   FUENTE          25 non-null     object 
 4   RUBRO           25 non-null     object 
 5   msb_c           25 non-null     float64
dtypes: float64(1), object(5)
memory usage: 1.3+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,msb_c
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1.0


In [196]:
ratios_sb_c = pd.merge(ratios_preliminar_red, sb_ratios_red, on=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], how='outer', validate = 'm:1')
postmerge(ratios_sb_c)
ratios_sb_c.info()
ratios_sb_c.head(88)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NIVEL_GOBIERNO     60 non-null     object 
 1   SECTOR             60 non-null     object 
 2   PLIEGO             60 non-null     object 
 3   FUENTE             60 non-null     object 
 4   RUBRO              60 non-null     object 
 5   CATEGORIA_GTO      59 non-null     object 
 6   GENERICA           59 non-null     object 
 7   mean_ratio_FONCOR  59 non-null     float64
 8   ratio_f_c          59 non-null     float64
 9   comp               59 non-null     float64
 10  msb_c              60 non-null     float64
dtypes: float64(4), object(7)
memory usage: 5.3+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,mean_ratio_FONCOR,ratio_f_c,comp,msb_c
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.003239,0.003239,0.000000e+00,1.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.996761,0.996761,0.000000e+00,1.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.000119,0.000119,0.000000e+00,1.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.999940,0.999881,0.000000e+00,1.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.001746,0.001745,0.000000e+00,1.0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.000899,0.000899,0.000000e+00,1.0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.997804,0.997356,0.000000e+00,1.0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTO CORRIENTE,3. BIENES Y SERVICIOS,0.051330,0.051363,0.000000e+00,1.0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.090239,0.090296,0.000000e+00,1.0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,6. GASTO DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.857792,0.858341,0.000000e+00,1.0


In [197]:
bd_sd.info()
bd_sd.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827 entries, 0 to 2826
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOB       2827 non-null   object 
 1   SECTOR          2827 non-null   object 
 2   PLIEGO          2827 non-null   object 
 3   EJECUTORA       2827 non-null   object 
 4   FUNCION         2827 non-null   object 
 5   FUENTE          2827 non-null   object 
 6   RUBRO           2827 non-null   object 
 7   CATEGORIA_GTO   2827 non-null   object 
 8   GENERICA        2827 non-null   object 
 9   SUBGENERICA     2827 non-null   object 
 10  PIA_2018        2827 non-null   int64  
 11  PIA_2019        2827 non-null   int64  
 12  PIA_2020        2827 non-null   int64  
 13  PIA_2021        2827 non-null   int64  
 14  PIA_2022        2827 non-null   int64  
 15  PIA_2023        2827 non-null   int64  
 16  PIA_2024        2827 non-null   int64  
 17  PIM_2018        2827 non-null   i

,NIVEL_GOB,SECTOR,PLIEGO,EJECUTORA,FUNCION,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,SUBGENERICA,...,PMG_2025,PMG_2026,PMG_2027,PMG_2028,PMG_2029,PMG_2030,PMG_2031,PMG_2032,PMG_2033,PMG_2034
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,"03. PLANEAMIENTO, GESTION Y RESERVA DE CONTING...",5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,1. AMORTIZACION DE LA DEUDA,...,0,0,0,0,0,0,0,0,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,25. DEUDA PUBLICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,1. AMORTIZACION DE LA DEUDA,...,0,0,0,0,0,0,0,0,0,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,25. DEUDA PUBLICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,2. INTERESES DE LA DEUDA,...,0,0,0,0,0,0,0,0,0,0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,25. DEUDA PUBLICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,3. COMISIONES Y OTROS GASTOS DE LA DEUDA,...,0,0,0,0,0,0,0,0,0,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,001. SEDE APURIMAC,25. DEUDA PUBLICA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",7. SERVICIO DE LA DEUDA,8. SERVICIO DE LA DEUDA PUBLICA,1. AMORTIZACION DE LA DEUDA,...,6418316,6418316,19470737,13052420,13052420,13052420,13052420,13052420,13052420,13052420


## Servicio de la deuda financiada con RD

In [198]:
serv_deuda = bd_sd.copy()

serv_deuda.rename(columns={'NIVEL_GOB' : 'NIVEL_GOBIERNO'}, inplace=True)
serv_deuda= serv_deuda[serv_deuda['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']
serv_deuda= serv_deuda[serv_deuda['FUENTE'] == '5. RECURSOS DETERMINADOS']
serv_deuda = serv_deuda[serv_deuda['RUBRO'].str[:2] == '15']
serv_deuda= serv_deuda[serv_deuda['GENERICA'] == '8. SERVICIO DE LA DEUDA PUBLICA']
# Calcular la suma de las columnas PMG_2025, PMG_2026 y PMG_2027 agrupadas por PLIEGO y GENERICA
serv_deuda = serv_deuda.groupby(['NIVEL_GOBIERNO','PLIEGO', 'GENERICA'])[['PMG_2025', 'PMG_2026', 'PMG_2027']].sum().reset_index()

# Renombrar las columnas PMG_2025, PMG_2026 y PMG_2027 a SD_2025_T, SD_2026_T y SD_2027_T respectivamente
serv_deuda = serv_deuda.rename(columns={'PMG_2025': 'SD_2025_T', 'PMG_2026': 'SD_2026_T', 'PMG_2027': 'SD_2027_T'})

# Reemplazar los valores nulos por 0 en las columnas especificadas
serv_deuda[['SD_2025_T', 'SD_2026_T', 'SD_2027_T']] = serv_deuda[['SD_2025_T', 'SD_2026_T', 'SD_2027_T']].fillna(0)

# Calcular la suma de las columnas SD_2025_T, SD_2026_T y SD_2027_T y guardarla en una nueva columna llamada SD_PMM
serv_deuda['SD_PMM'] = serv_deuda[['SD_2025_T', 'SD_2026_T', 'SD_2027_T']].sum(axis=1)

# Eliminar filas donde SD_PMM es igual a 0 o es nulo
serv_deuda = serv_deuda.dropna(subset=['SD_PMM'])
serv_deuda = serv_deuda[serv_deuda['SD_PMM'].fillna(0) != 0].reset_index(drop=True)

serv_deuda.info()
serv_deuda.head(16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  0 non-null      object
 1   PLIEGO          0 non-null      object
 2   GENERICA        0 non-null      object
 3   SD_2025_T       0 non-null      int64 
 4   SD_2026_T       0 non-null      int64 
 5   SD_2027_T       0 non-null      int64 
 6   SD_PMM          0 non-null      int64 
dtypes: int64(4), object(3)
memory usage: 132.0+ bytes


,NIVEL_GOBIERNO,PLIEGO,GENERICA,SD_2025_T,SD_2026_T,SD_2027_T,SD_PMM


In [199]:
bd_rubro15.info()
bd_rubro15.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NIVEL_GOBIERNO    26 non-null     object
 1   SECTOR            26 non-null     object
 2   PLIEGO            26 non-null     object
 3   EJECUTORA         26 non-null     object
 4   FUENTE            26 non-null     object
 5   RUBRO             26 non-null     object
 6   tipo_gen          26 non-null     object
 7   ETIQUETA_RD       26 non-null     object
 8   RECAUDACION_2024  26 non-null     int64 
 9   RECAUDACION_2025  26 non-null     int64 
 10  RECAUDACION_2026  26 non-null     int64 
 11  RECAUDACION_2027  26 non-null     int64 
 12  PMG_2025_gg3_1    26 non-null     int64 
 13  PMG_2025_gg6_2    26 non-null     int64 
 14  PMG_2025_gg7_2    26 non-null     int64 
 15  PMG_2026_gg3_1    26 non-null     int64 
 16  PMG_2026_gg6_2    26 non-null     int64 
 17  PMG_2026_gg7_2    

,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,FUENTE,RUBRO,tipo_gen,ETIQUETA_RD,RECAUDACION_2024,RECAUDACION_2025,...,RECAUDACION_2027,PMG_2025_gg3_1,PMG_2025_gg6_2,PMG_2025_gg7_2,PMG_2026_gg3_1,PMG_2026_gg6_2,PMG_2026_gg7_2,PMG_2027_gg3_1,PMG_2027_gg6_2,PMG_2027_gg7_2
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,9. Foncor_SB,15000000,15000000,...,15000000,0,0,0,0,0,0,0,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,001. SEDE APURIMAC,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,9. Foncor_SB,289700,300000,...,400000,0,0,0,0,0,0,0,0,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,002. SEDE CHANKA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,9. Foncor_SB,158891,150000,...,150000,0,0,0,0,0,0,0,0,0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,001. SEDE AREQUIPA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,9. Foncor_SB,35000000,41000000,...,41000000,0,0,0,0,0,0,0,0,0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,001. SEDE AYACUCHO,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,9. Foncor_SB,0,1000000,...,1200000,0,0,1000000,0,0,1100000,0,0,1200000


## Base ingresos Saldos de balance RD 

In [200]:
PIASB2527 = bd_rubro15.copy()

# Mantener solo las filas donde la columna tipo_gen es igual a "1. SALDOS DE BALANCE"
PIASB2527 = PIASB2527[PIASB2527['tipo_gen'] == "1. SALDOS DE BALANCE"]
PIASB2527= PIASB2527[PIASB2527['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']
PIASB2527['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"

# Renombrar las columnas RECAUDACION_2025, RECAUDACION_2026 y RECAUDACION_2027 a PMG_2025, PMG_2026 y PMG_2027 respectivamente
PIASB2527.rename(columns={'RECAUDACION_2025': 'PMG_2025', 'RECAUDACION_2026': 'PMG_2026', 'RECAUDACION_2027': 'PMG_2027'}, inplace=True)

# Agrupar por las columnas especificadas y sumar las columnas PMG_2025, PMG_2026 y PMG_2027
PIASB2527 = PIASB2527.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'tipo_gen']).agg({'PMG_2025': 'sum', 'PMG_2026': 'sum', 'PMG_2027': 'sum', 
                                                                                                             'PMG_2025_gg3_1': 'sum', 'PMG_2025_gg6_2':'sum',	'PMG_2025_gg7_2': 'sum', 
                                                                                                             'PMG_2026_gg3_1': 'sum', 'PMG_2026_gg6_2': 'sum', 'PMG_2026_gg7_2': 'sum', 
                                                                                                             'PMG_2027_gg3_1': 'sum', 'PMG_2027_gg6_2': 'sum', 'PMG_2027_gg7_2': 'sum'}).reset_index()

# Renombrar las columnas PMG_2025, PMG_2026 y PMG_2027 a SB_2025, SB_2026, SB_2027 y tipo_gen a SaldoBalance respectivamente
PIASB2527.rename(columns={'PMG_2025': 'SB_2025', 'PMG_2026': 'SB_2026', 'PMG_2027': 'SB_2027', 'tipo_gen': 'SaldoBalance'}, inplace=True)

# Redondear los valores en las columnas SB_2025, SB_2026 y SB_2027
PIASB2527['SB_2025'] = PIASB2527['SB_2025'].round()
PIASB2527['SB_2026'] = PIASB2527['SB_2026'].round()
PIASB2527['SB_2027'] = PIASB2527['SB_2027'].round()

PIASB2527.info()
PIASB2527.head(24)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  15 non-null     object
 1   SECTOR          15 non-null     object
 2   PLIEGO          15 non-null     object
 3   FUENTE          15 non-null     object
 4   RUBRO           15 non-null     object
 5   SaldoBalance    15 non-null     object
 6   SB_2025         15 non-null     int64 
 7   SB_2026         15 non-null     int64 
 8   SB_2027         15 non-null     int64 
 9   PMG_2025_gg3_1  15 non-null     int64 
 10  PMG_2025_gg6_2  15 non-null     int64 
 11  PMG_2025_gg7_2  15 non-null     int64 
 12  PMG_2026_gg3_1  15 non-null     int64 
 13  PMG_2026_gg6_2  15 non-null     int64 
 14  PMG_2026_gg7_2  15 non-null     int64 
 15  PMG_2027_gg3_1  15 non-null     int64 
 16  PMG_2027_gg6_2  15 non-null     int64 
 17  PMG_2027_gg7_2  15 non-null     int64 
dtypes: int64(12)

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,SaldoBalance,SB_2025,SB_2026,SB_2027,PMG_2025_gg3_1,PMG_2025_gg6_2,PMG_2025_gg7_2,PMG_2026_gg3_1,PMG_2026_gg6_2,PMG_2026_gg7_2,PMG_2027_gg3_1,PMG_2027_gg6_2,PMG_2027_gg7_2
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,15000000,15000000,15000000,0,0,0,0,0,0,0,0,0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,450000,539700,550000,0,0,0,0,0,0,0,0,0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,41000000,41000000,41000000,0,0,0,0,0,0,0,0,0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,1000000,1100000,1200000,0,0,1000000,0,0,1100000,0,0,1200000
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,8135129,8343833,8748253,8135129,0,0,8343833,0,0,8748253,0,0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,100000,100000,100000,0,0,0,0,0,0,0,0,0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,57629700,57629700,57629700,7629700,48000000,0,7629700,48000000,0,7629700,48000000,0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,99441011,99441011,99489678,0,0,0,0,0,0,0,0,0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,451. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA ...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,76388115,76388115,76388115,0,1951393,0,0,1951393,0,0,1951393,0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,454. GOBIERNO REGIONAL DEL DEPARTAMENTO DE MAD...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,262244,314694,377632,0,0,0,0,0,0,0,0,0


## Base de Estimaciones a los recursos de RD + Saldos de Balance  

In [201]:
R18= bd_R18.copy()
R18=R18[['NIVEL DE GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'ESPECIFICA DET', '2025', '2026', '2027']]
R18.rename(columns={'NIVEL DE GOBIERNO': 'NIVEL_GOBIERNO'}, inplace=True)
R18= R18[R18['NIVEL_GOBIERNO'] == '2. GOBIERNOS REGIONALES']
R18= R18[R18['FUENTE'] == '5. RECURSOS DETERMINADOS']
R18= R18[R18['RUBRO'] == '15. FONDO DE COMPENSACION REGIONAL - FONCOR']
R18 = R18.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'ESPECIFICA DET']).agg({'2025':'sum', '2026':'sum', '2027':'sum'}).reset_index()
# Eliminar filas duplicadas en el dataframe
R18.drop_duplicates(inplace=True)
R18.info()
R18.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOBIERNO  23 non-null     object 
 1   SECTOR          23 non-null     object 
 2   PLIEGO          23 non-null     object 
 3   FUENTE          23 non-null     object 
 4   RUBRO           23 non-null     object 
 5   ESPECIFICA DET  23 non-null     object 
 6   2025            23 non-null     int64  
 7   2026            23 non-null     float64
 8   2027            23 non-null     float64
dtypes: float64(2), int64(1), object(6)
memory usage: 1.7+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,ESPECIFICA DET,2025,2026,2027
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. FONDO DE COMPENSACION REGIONAL,233549217,239184885.0,247521649.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. FONDO DE COMPENSACION REGIONAL,169753667,180776045.0,197139809.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. FONDO DE COMPENSACION REGIONAL,97313757,101018446.0,106510198.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. FONDO DE COMPENSACION REGIONAL,352580607,365157492.0,383796675.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. FONDO DE COMPENSACION REGIONAL,236318662,243050883.0,253018460.0


In [202]:
bd_R18_25= R18.copy()
bd_R18_25 = bd_R18_25.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], columns='ESPECIFICA DET', values=['2025']).reset_index().rename_axis(None)
bd_R18_25.columns = [f'{i}{j}' for i, j in bd_R18_25.columns]

bd_R18_25.rename(columns={'20251. FONDO DE COMPENSACION REGIONAL':'FONCOR'}, inplace=True)

bd_R18_25=bd_R18_25[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'FONCOR']]
bd_R18_25.fillna(0, inplace=True)

variables = ['FONCOR']
for var in variables:
    bd_R18_25[var] = bd_R18_25[var].round()
    bd_R18_25[var] =  bd_R18_25[var].astype(int)
    
bd_R18_25.info()
bd_R18_25.head(51)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  23 non-null     object
 1   SECTOR          23 non-null     object
 2   PLIEGO          23 non-null     object
 3   FUENTE          23 non-null     object
 4   RUBRO           23 non-null     object
 5   FONCOR          23 non-null     int32 
dtypes: int32(1), object(5)
memory usage: 1.1+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,233549217
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,169753667
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,97313757
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,352580607
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,236318662
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,580202373
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,113793970
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,375036067
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,214575762
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,576784786


In [203]:
bd_R18_26= R18.copy()
bd_R18_26 = bd_R18_26.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], columns='ESPECIFICA DET', values=['2026']).reset_index().rename_axis(None)
bd_R18_26.columns = [f'{i}{j}' for i, j in bd_R18_26.columns]

bd_R18_26.rename(columns={'20261. FONDO DE COMPENSACION REGIONAL':'FONCOR'}, inplace=True)

bd_R18_26=bd_R18_26[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'FONCOR']]
bd_R18_26.fillna(0, inplace=True)

variables = ['FONCOR']
for var in variables:
    bd_R18_26[var] = bd_R18_26[var].round()
    bd_R18_26[var] =  bd_R18_26[var].astype(int)
    
bd_R18_26.info()
bd_R18_26.head(51)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  23 non-null     object
 1   SECTOR          23 non-null     object
 2   PLIEGO          23 non-null     object
 3   FUENTE          23 non-null     object
 4   RUBRO           23 non-null     object
 5   FONCOR          23 non-null     int32 
dtypes: int32(1), object(5)
memory usage: 1.1+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,239184885
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,180776045
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,101018446
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,365157492
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,243050883
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,594773377
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,117408642
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,383742958
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,223197246
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,590958072


In [204]:
bd_R18_27= R18.copy()
bd_R18_27 = bd_R18_27.pivot_table(index=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO'], columns='ESPECIFICA DET', values=['2027']).reset_index().rename_axis(None)
bd_R18_27.columns = [f'{i}{j}' for i, j in bd_R18_27.columns]

bd_R18_27.rename(columns={'20271. FONDO DE COMPENSACION REGIONAL':'FONCOR'}, inplace=True)

bd_R18_27=bd_R18_27[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'FONCOR']]
bd_R18_27.fillna(0, inplace=True)

variables = ['FONCOR']
for var in variables:
    bd_R18_27[var] = bd_R18_27[var].round()
    bd_R18_27[var] =  bd_R18_27[var].astype(int)
    
bd_R18_27.info()
bd_R18_27.head(51)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  23 non-null     object
 1   SECTOR          23 non-null     object
 2   PLIEGO          23 non-null     object
 3   FUENTE          23 non-null     object
 4   RUBRO           23 non-null     object
 5   FONCOR          23 non-null     int32 
dtypes: int32(1), object(5)
memory usage: 1.1+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,247521649
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,197139809
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,106510198
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,383796675
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,253018460
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,616332873
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,122763115
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,396620038
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,235979979
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,611926567


In [205]:
R18_25 = bd_R18_25.copy() 
R18_26 = bd_R18_26.copy() 
R18_27 = bd_R18_27.copy() 

In [206]:
R18_25.info()
R18_25.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  23 non-null     object
 1   SECTOR          23 non-null     object
 2   PLIEGO          23 non-null     object
 3   FUENTE          23 non-null     object
 4   RUBRO           23 non-null     object
 5   FONCOR          23 non-null     int32 
dtypes: int32(1), object(5)
memory usage: 1.1+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,233549217
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,169753667
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,97313757
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,352580607
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,236318662


In [207]:
R18_26.info()
R18_26.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  23 non-null     object
 1   SECTOR          23 non-null     object
 2   PLIEGO          23 non-null     object
 3   FUENTE          23 non-null     object
 4   RUBRO           23 non-null     object
 5   FONCOR          23 non-null     int32 
dtypes: int32(1), object(5)
memory usage: 1.1+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,239184885
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,180776045
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,101018446
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,365157492
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,243050883


In [208]:
R18_27.info()
R18_27.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  23 non-null     object
 1   SECTOR          23 non-null     object
 2   PLIEGO          23 non-null     object
 3   FUENTE          23 non-null     object
 4   RUBRO           23 non-null     object
 5   FONCOR          23 non-null     int32 
dtypes: int32(1), object(5)
memory usage: 1.1+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,FONCOR
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,247521649
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,197139809
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,106510198
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,383796675
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,253018460
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,616332873
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,122763115
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,396620038
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,235979979
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,611926567


In [209]:
def PIA2527(df,year):
    
#    df.rename(columns={"CANON MINERO": "CANONMINERO",
 #                      "CANON PETROLERO": "CANONPETROLERO",
  #                     "SOBRECANON PETROLERO": "SOBRECANONPETROLERO",
   #                    "CANON GASIFERO": "CANONGASIFERO",
    #                   "CANON HIDROENER-GETICO": "CANONHIDROENERGETICO",
     #                  "CANON PESQUERO": "CANONPESQUERO",
      #                 "CANON FORESTAL": "CANONFORESTAL",
       #                "REGALIAS MINERAS": "REGALIASMINERAS",
        #               "ELIMINACION  EXONERACIONES E INCENTIVOS TRIBUTARIOS": "ELIMINACIONEXONERACIONESEINC",
         #              "REGALIA CONTRACTUAL": "REGALIACONTRACTUAL",
          #             "FOCAM ":"FOCAM",
            #           'RENTA DE ADUANAS  1/':'RENTADEADUANAS1'}, inplace=True)
    
    variables = ['FONCOR']
    
    for var in variables: 
        df[var] = df[var].astype(str)
        df[var] = df[var].str.replace(" ", '0')
        df[var] = df[var].fillna('0')
        df[var] =  df[var].astype(int)
    
    # Filtrar por "2. GOBIERNOS REGIONALES"
    df = df[df["NIVEL_GOBIERNO"] == "2. GOBIERNOS REGIONALES"]
    

    # Convertir columnas a números y reemplazar valores no válidos
    df["FONCOR"] = pd.to_numeric(df["FONCOR"], errors="coerce")
    #df["REGALIACONTRACTUAL"] = pd.to_numeric(df["REGALIACONTRACTUAL"], errors="coerce")
    df.dropna(subset=["FONCOR"], inplace=True)

    # Sumarizar y colapsar
    df = df.groupby("PLIEGO").agg({'NIVEL_GOBIERNO':'first', "FONCOR": "sum"}).reset_index()

    # Redondear variables específicas
    #vars_to_round = ["CANONMINERO", "CANONPETROLERO", "SOBRECANONPETROLERO", "CANONGASIFERO", 
     #                "CANONHIDROENERGETICO", "CANONPESQUERO", "CANONFORESTAL", 'RENTADEADUANAS1', "FOCAM", 
      #               "REGALIASMINERAS", "ELIMINACIONEXONERACIONESEINC", "REGALIACONTRACTUAL"]
    #df[vars_to_round] = df[vars_to_round].applymap(lambda x: int(x) if pd.notnull(x) else x)

    # Renombrar columnas
    #df.rename(columns={"FOCAM": "Regalias_FOCAM"}, inplace=True)

    # Calcular la suma de las columnas especificadas y almacenarla en una nueva columna
    
    df['Foncor'] = df[["FONCOR"]].sum(axis=1)
    
    df["Res_Direct"] = df[["FONCOR"]].sum(axis=1)
    ##verificar que Res_Direct sean apropiadas las variables que se suman ???.
    
    # Renombrar columnas
 #   df.rename(columns={"REGALIASMINERAS": "Regalias_Mineras", 
  #                     "ELIMINACIONEXONERACIONESEINC": "Participación_Elim_Exo",
   #                    "REGALIACONTRACTUAL": "Regalias_Contractuales", 
    #                   'RENTADEADUANAS1' : 'Participación_Renta_Aduana'}, inplace=True)

    # Eliminar columnas innecesarias
    df.drop(columns=['FONCOR'], inplace=True)

    # Fusionar con otro DataFrame
    #*Incluimos saldos de balance del mòdulo de programacion de ingresos
    df = df.merge(PIASB2527[["PLIEGO", f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2"]], on="PLIEGO", how="left", validate = '1:1')
    postmerge(df)
    df[f"SB_{year}"].fillna(0, inplace=True)
    df[f"PMG_{year}_gg3_1"].fillna(0, inplace=True)
    df[f"PMG_{year}_gg6_2"].fillna(0, inplace=True)
    
    # Calcular PIA_BASE
    df["PIA_BASE"] = df["Res_Direct"].fillna(0) + df[f"SB_{year}"].fillna(0)

    # Agregar columna AÑO
    df["AÑO"] = year
    df['SECTOR'] = "99. GOBIERNOS REGIONALES"
    df['FUENTE'] = "5. RECURSOS DETERMINADOS"
    df['RUBRO'] = "15. FONDO DE COMPENSACION REGIONAL - FONCOR"
    df['NIVEL_GOBIERNO'] = "2. GOBIERNOS REGIONALES"
    # Renombrar columnas SB*
    df.rename(columns={"PIA_BASE":f"PIA_BASE_{year}"}, inplace=True)
    return df

In [210]:
PIA_2025_base24 = PIA2527(R18_25, 2025)
PIA_2025_base24.info()
PIA_2025_base24.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          23 non-null     object 
 1   NIVEL_GOBIERNO  23 non-null     object 
 2   Foncor          23 non-null     int64  
 3   Res_Direct      23 non-null     int64  
 4   SB_2025         23 non-null     float64
 5   PMG_2025_gg3_1  23 non-null     float64
 6   PMG_2025_gg6_2  23 non-null     float64
 7   PIA_BASE_2025   23 non-null     float64
 8   AÑO             23 non-null     int64  
 9   SECTOR          23 non-null     object 
 10  FUENTE          23 non-null     object 
 11  RUBRO           23 non-null     object 
dtypes: float64(4), int64(3), object(5)
memory usage: 2.3+ KB


,PLIEGO,NIVEL_GOBIERNO,Foncor,Res_Direct,SB_2025,PMG_2025_gg3_1,PMG_2025_gg6_2,PIA_BASE_2025,AÑO,SECTOR,FUENTE,RUBRO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,2. GOBIERNOS REGIONALES,233549217,233549217,0.0,0.0,0.0,233549217.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
1,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,2. GOBIERNOS REGIONALES,169753667,169753667,15000000.0,0.0,0.0,184753667.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
2,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,2. GOBIERNOS REGIONALES,97313757,97313757,450000.0,0.0,0.0,97763757.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
3,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,2. GOBIERNOS REGIONALES,352580607,352580607,41000000.0,0.0,0.0,393580607.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
4,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,2. GOBIERNOS REGIONALES,236318662,236318662,1000000.0,0.0,0.0,237318662.0,2025,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR


In [211]:
PIA_2026_base24 = PIA2527(R18_26, 2026)
PIA_2026_base24.info()
PIA_2026_base24.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          23 non-null     object 
 1   NIVEL_GOBIERNO  23 non-null     object 
 2   Foncor          23 non-null     int64  
 3   Res_Direct      23 non-null     int64  
 4   SB_2026         23 non-null     float64
 5   PMG_2026_gg3_1  23 non-null     float64
 6   PMG_2026_gg6_2  23 non-null     float64
 7   PIA_BASE_2026   23 non-null     float64
 8   AÑO             23 non-null     int64  
 9   SECTOR          23 non-null     object 
 10  FUENTE          23 non-null     object 
 11  RUBRO           23 non-null     object 
dtypes: float64(4), int64(3), object(5)
memory usage: 2.3+ KB


,PLIEGO,NIVEL_GOBIERNO,Foncor,Res_Direct,SB_2026,PMG_2026_gg3_1,PMG_2026_gg6_2,PIA_BASE_2026,AÑO,SECTOR,FUENTE,RUBRO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,2. GOBIERNOS REGIONALES,239184885,239184885,0.0,0.0,0.0,239184885.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
1,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,2. GOBIERNOS REGIONALES,180776045,180776045,15000000.0,0.0,0.0,195776045.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
2,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,2. GOBIERNOS REGIONALES,101018446,101018446,539700.0,0.0,0.0,101558146.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
3,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,2. GOBIERNOS REGIONALES,365157492,365157492,41000000.0,0.0,0.0,406157492.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
4,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,2. GOBIERNOS REGIONALES,243050883,243050883,1100000.0,0.0,0.0,244150883.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
5,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,2. GOBIERNOS REGIONALES,594773377,594773377,8343833.0,8343833.0,0.0,603117210.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
6,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,2. GOBIERNOS REGIONALES,117408642,117408642,100000.0,0.0,0.0,117508642.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
7,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,2. GOBIERNOS REGIONALES,383742958,383742958,57629700.0,7629700.0,48000000.0,441372658.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
8,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,2. GOBIERNOS REGIONALES,223197246,223197246,99441011.0,0.0,0.0,322638257.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
9,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,2. GOBIERNOS REGIONALES,590958072,590958072,0.0,0.0,0.0,590958072.0,2026,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR


In [212]:
PIA_2027_base24 = PIA2527(R18_27, 2027)
PIA_2027_base24.info()
PIA_2027_base24.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          23 non-null     object 
 1   NIVEL_GOBIERNO  23 non-null     object 
 2   Foncor          23 non-null     int64  
 3   Res_Direct      23 non-null     int64  
 4   SB_2027         23 non-null     float64
 5   PMG_2027_gg3_1  23 non-null     float64
 6   PMG_2027_gg6_2  23 non-null     float64
 7   PIA_BASE_2027   23 non-null     float64
 8   AÑO             23 non-null     int64  
 9   SECTOR          23 non-null     object 
 10  FUENTE          23 non-null     object 
 11  RUBRO           23 non-null     object 
dtypes: float64(4), int64(3), object(5)
memory usage: 2.3+ KB


,PLIEGO,NIVEL_GOBIERNO,Foncor,Res_Direct,SB_2027,PMG_2027_gg3_1,PMG_2027_gg6_2,PIA_BASE_2027,AÑO,SECTOR,FUENTE,RUBRO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,2. GOBIERNOS REGIONALES,247521649,247521649,0.0,0.0,0.0,2.475216e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
1,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,2. GOBIERNOS REGIONALES,197139809,197139809,15000000.0,0.0,0.0,2.121398e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
2,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,2. GOBIERNOS REGIONALES,106510198,106510198,550000.0,0.0,0.0,1.070602e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
3,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,2. GOBIERNOS REGIONALES,383796675,383796675,41000000.0,0.0,0.0,4.247967e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
4,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,2. GOBIERNOS REGIONALES,253018460,253018460,1200000.0,0.0,0.0,2.542185e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
5,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,2. GOBIERNOS REGIONALES,616332873,616332873,8748253.0,8748253.0,0.0,6.250811e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
6,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,2. GOBIERNOS REGIONALES,122763115,122763115,100000.0,0.0,0.0,1.228631e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
7,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,2. GOBIERNOS REGIONALES,396620038,396620038,57629700.0,7629700.0,48000000.0,4.542497e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
8,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,2. GOBIERNOS REGIONALES,235979979,235979979,99489678.0,0.0,0.0,3.354697e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
9,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,2. GOBIERNOS REGIONALES,611926567,611926567,0.0,0.0,0.0,6.119266e+08,2027,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR


# 3. Propuesta nueva RD(modif) PMG 2025 -2027

## DISTRIBUCIÓN 2025, 2026 y 2027

### Generar PIA Base de ingresos sobre el cual se hace la distribución de gastos  (SB y Flujo)

In [213]:
PIA_2025_bd24= PIA_2025_base24.copy()
PIA_2026_bd24= PIA_2026_base24.copy()
PIA_2027_bd24= PIA_2027_base24.copy()

### 7. SERVICIO DE LA DEUDA

In [214]:
def PIASBF(df,year):
    
    variables = ['Foncor']
    
    for var in variables: 
        df[var] = df[var].fillna(0)
        
    # Calcular la suma de las columnas especificadas y almacenarla en una nueva columna
    
    df['C_FONCOR'] = df[["Foncor"]].sum(axis=1) 
  #  df['OR'] = df[["Regalias_FOCAM", "Participación_Elim_Exo", "Regalias_Contractuales" ]].sum(axis=1)
   # df['RA'] = df['Participación_Renta_Aduana']
    
    # Eliminar columnas innecesarias
    df.drop(columns=['Foncor'], inplace=True)
    df = df[['AÑO', 'PLIEGO', 'C_FONCOR', 'Res_Direct', f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2", f"PIA_BASE_{year}"]]
    
    #Distribución RD entre Genéricas de Gasto 
    #FINANCIO GG 2.8 EN PRIMER LUGAR (no se usa saldo de balance para cubrir servicio de la deuda)
    #// Información de servicio de la deuda
    df = df.merge(serv_deuda[["PLIEGO", f"SD_{year}_T"]], on="PLIEGO", how="left", validate = '1:1')
    postmerge(df)
    df[f"SD_{year}_T"] = df[f"SD_{year}_T"].round()
    df[f"SD_{year}_T"] = df[f"SD_{year}_T"].fillna(0)
    
    #Financio SD con Canon_Sobrecanon y regalías Mineras (// deuda financiada con Canon sobre canon y regalias mineras)
    df['C_FONCOR_sd'] =np.nan
        
    # Reemplazar valores en C_SC_RM_sd basados en una condición
    df['C_FONCOR_sd'] = np.where((df[f"SD_{year}_T"] < df['C_FONCOR']), df[f"SD_{year}_T"],
                               np.where(((df['C_FONCOR'].fillna(0) != 0) & (df['C_FONCOR_sd'].isna()) & (df[f"SD_{year}_T"].fillna(0) != 0)), df['C_FONCOR'], df['C_FONCOR_sd']))
    
    #// identificamos deuda pendiente de financiar con otro rubro
    # Ordenar el DataFrame por la columna PLIEGO
    df = df.sort_values(by='PLIEGO')

    # Calcular la suma acumulativa de C_SC_RM_sd para cada grupo de PLIEGO
    df['t2'] = df.groupby('PLIEGO')['C_FONCOR_sd'].transform('sum')

    # Calcular la diferencia entre SD_20x_T y la suma acumulativa t2
    df['SD4'] = df[f"SD_{year}_T"] - df['t2']
    #*2. Financio SD con Otros ( // deuda financiada con otras fuentes (foncan, exoneraciones))
    #df = df.assign(OR_sd = np.nan)
    #df['OR_sd'] = np.where((df['SD4']<df['OR']), df['SD4'],
                         #  np.where(((df['OR'].fillna(0) != 0) & (df['OR_sd'].isna()) & (df['SD4'].fillna(0) != 0)), df['OR'], df['OR_sd']))      
    #// identificamos deuda pendiente de financiar con otro rubro
    #df['t4'] = df.groupby('PLIEGO')['OR_sd'].transform('sum')
    #df['SD5'] = df['SD4'] - df['t4']
    #*3. Financio SD con Participación_Renta_Aduana ( // deuda financiada con renta de aduana)
    #df = df.assign(RA_sd = np.nan)
    #df['RA_sd'] = np.where((df['SD5']<df['RA']), df['SD5'],
                       #    np.where(((df['RA'].fillna(0) != 0) & (df['RA_sd'].isna()) & (df['SD5'].fillna(0)!= 0)), df['RA'], df['RA_sd'])) 
    #// identificamos deuda que no se financiarà
    #df['t5'] = df.groupby('PLIEGO')['RA_sd'].transform('sum')
    #df['SD6'] = df['SD5'] - df['t5']
    
    # Eliminar columnas innecesarias
    #df.drop(columns=[ "t2", "t4", "t5", "SD4", "SD5", "SD6"], inplace=True)
    
    variables = ['C_FONCOR_sd']
    
    for var in variables: 
        df[var] = df[var].fillna(0)
    # Generando ratios de la propuesta para financiar todo el SD (Tesoro) con RD: pago realizado vs recurso total
    df['ratio_t_cfoncor'] = df['C_FONCOR_sd']/df['C_FONCOR']
    #df['ratio_t_or'] = df['OR_sd']/df['OR']
    #df['ratio_t_ra'] = df['RA_sd']/df['RA']
    #Ya no se promedio con el ratio de ejecución promedio dado que se decidió priorizar el pago de la deuda 
    #(Uso los ratios totales de la propuesta de DGETP no los finales promedio.)
    # Calcular las variables ajustadas
    df['C_FONCOR_a'] = round(df['C_FONCOR'] * df['ratio_t_cfoncor'])
    df['C_FONCOR_a'].fillna(0, inplace=True)

    #df['OR_a'] = round(df['OR'] * df['ratio_t_or'])
    #df['OR_a'].fillna(0, inplace=True)

    #df['RA_a'] = round(df['RA'] * df['ratio_t_ra'])
    #df['RA_a'].fillna(0, inplace=True)

    # Calcular SD_ax
    df[f"SD_a{year}"] = df[["C_FONCOR_a"]].sum(axis=1) 

    # Eliminar las columnas especificadas
    columns_to_drop = ['C_FONCOR_sd', 'ratio_t_cfoncor']
    df.drop(columns=columns_to_drop, inplace=True)
    
    ######################*Para appender a la otrs GGs
    # Agrupar y sumar las variables por PLIEGO
    df = df.groupby('PLIEGO').agg({'C_FONCOR_a': 'sum',  f"SD_a{year}": 'sum'}).reset_index()

    # Renombrar columnas
    df.rename(columns={'C_FONCOR_a': 'PR_C_FONCOR', f"SD_a{year}": 'MONTOSD_RD'}, inplace=True)

    # Agregar columnas GENERICA, CATEGORIA_GASTO, SECTOR, FUENTE y RUBRO
    df['GENERICA'] = "8. SERVICIO DE LA DEUDA PUBLICA"
    df['CATEGORIA_GTO'] = "7. SERVICIO DE LA DEUDA"
    df['SECTOR'] = "99. GOBIERNOS REGIONALES"
    df['FUENTE'] = "5. RECURSOS DETERMINADOS"
    df['RUBRO'] = "15. FONDO DE COMPENSACION REGIONAL - FONCOR"
    df['NIVEL_GOBIERNO'] = "2. GOBIERNOS REGIONALES"

    # Eliminar filas donde MONTOSD_RD es igual a 0
    df = df[df['MONTOSD_RD'].fillna(0) != 0].reset_index(drop=True)
    return df

In [215]:
SDT_2025=PIASBF(PIA_2025_bd24,2025)
SDT_2025.info()
SDT_2025.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          0 non-null      object 
 1   PR_C_FONCOR     0 non-null      float64
 2   MONTOSD_RD      0 non-null      float64
 3   GENERICA        0 non-null      object 
 4   CATEGORIA_GTO   0 non-null      object 
 5   SECTOR          0 non-null      object 
 6   FUENTE          0 non-null      object 
 7   RUBRO           0 non-null      object 
 8   NIVEL_GOBIERNO  0 non-null      object 
dtypes: float64(2), object(7)
memory usage: 132.0+ bytes


,PLIEGO,PR_C_FONCOR,MONTOSD_RD,GENERICA,CATEGORIA_GTO,SECTOR,FUENTE,RUBRO,NIVEL_GOBIERNO


In [216]:
SDT_2026=PIASBF(PIA_2026_bd24,2026)
SDT_2026.info()
SDT_2026.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          0 non-null      object 
 1   PR_C_FONCOR     0 non-null      float64
 2   MONTOSD_RD      0 non-null      float64
 3   GENERICA        0 non-null      object 
 4   CATEGORIA_GTO   0 non-null      object 
 5   SECTOR          0 non-null      object 
 6   FUENTE          0 non-null      object 
 7   RUBRO           0 non-null      object 
 8   NIVEL_GOBIERNO  0 non-null      object 
dtypes: float64(2), object(7)
memory usage: 132.0+ bytes


,PLIEGO,PR_C_FONCOR,MONTOSD_RD,GENERICA,CATEGORIA_GTO,SECTOR,FUENTE,RUBRO,NIVEL_GOBIERNO


In [217]:
SDT_2027=PIASBF(PIA_2027_bd24,2027)
SDT_2027.info()
SDT_2027.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          0 non-null      object 
 1   PR_C_FONCOR     0 non-null      float64
 2   MONTOSD_RD      0 non-null      float64
 3   GENERICA        0 non-null      object 
 4   CATEGORIA_GTO   0 non-null      object 
 5   SECTOR          0 non-null      object 
 6   FUENTE          0 non-null      object 
 7   RUBRO           0 non-null      object 
 8   NIVEL_GOBIERNO  0 non-null      object 
dtypes: float64(2), object(7)
memory usage: 132.0+ bytes


,PLIEGO,PR_C_FONCOR,MONTOSD_RD,GENERICA,CATEGORIA_GTO,SECTOR,FUENTE,RUBRO,NIVEL_GOBIERNO


### PIA reducido (menos los recursos para la GG 2.8)

In [218]:
PIA_2025_bd24= PIA_2025_base24.copy()
PIA_2026_bd24= PIA_2026_base24.copy()
PIA_2027_bd24= PIA_2027_base24.copy()

In [219]:
def PIASBF1(df,year):
    
    variables = ['Foncor']
    
    for var in variables: 
        df[var] = df[var].fillna(0)
        
    # Calcular la suma de las columnas especificadas y almacenarla en una nueva columna
    
    df['C_FONCOR'] = df[["Foncor"]].sum(axis=1) 
  #  df['OR'] = df[["Regalias_FOCAM", "Participación_Elim_Exo", "Regalias_Contractuales" ]].sum(axis=1)
   # df['RA'] = df['Participación_Renta_Aduana']
    
    # Eliminar columnas innecesarias
    df.drop(columns=['Foncor'], inplace=True)
    df = df[['AÑO', 'PLIEGO', 'C_FONCOR','Res_Direct', f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2",  f"PIA_BASE_{year}"]]
    
    #Distribución RD entre Genéricas de Gasto 
    #FINANCIO GG 2.8 EN PRIMER LUGAR (no se usa saldo de balance para cubrir servicio de la deuda)
    #// Información de servicio de la deuda
    df = df.merge(serv_deuda[["PLIEGO", f"SD_{year}_T"]], on="PLIEGO", how="left", validate = '1:1')
    postmerge(df)
    df[f"SD_{year}_T"] = df[f"SD_{year}_T"].round()
    df[f"SD_{year}_T"] = df[f"SD_{year}_T"].fillna(0)
    
    #Financio SD con Canon_Sobrecanon y regalías Mineras (// deuda financiada con Canon sobre canon y regalias mineras)
    df['C_FONCOR_sd'] =np.nan
        
    # Reemplazar valores en C_SC_RM_sd basados en una condición
    df['C_FONCOR_sd'] = np.where((df[f"SD_{year}_T"] < df['C_FONCOR']), df[f"SD_{year}_T"],
                               np.where(((df['C_FONCOR'] != 0) & (df['C_FONCOR_sd'].isna()) & (df[f"SD_{year}_T"] != 0)), df['C_FONCOR'], df['C_FONCOR_sd']))
    
    #// identificamos deuda pendiente de financiar con otro rubro
    # Ordenar el DataFrame por la columna PLIEGO
    df = df.sort_values(by='PLIEGO')

    # Calcular la suma acumulativa de C_SC_RM_sd para cada grupo de PLIEGO
    df['t2'] = df.groupby('PLIEGO')['C_FONCOR_sd'].transform('sum')

    # Calcular la diferencia entre SD_20x_T y la suma acumulativa t2
    df['SD4'] = df[f"SD_{year}_T"] - df['t2']
    #*2. Financio SD con Otros ( // deuda financiada con otras fuentes (foncan, exoneraciones))
   # df = df.assign(OR_sd = np.nan)
   # df['OR_sd'] = np.where((df['SD4']<df['OR']), df['SD4'],
                         #  np.where(((df['OR'] != 0) & (df['OR_sd'].isna()) & (df['SD4']!= 0)), df['OR'], df['OR_sd']))      
    #// identificamos deuda pendiente de financiar con otro rubro
    #df['t4'] = df.groupby('PLIEGO')['OR_sd'].transform('sum')
    #df['SD5'] = df['SD4'] - df['t4']
    #*3. Financio SD con Participación_Renta_Aduana ( // deuda financiada con renta de aduana)
    #df = df.assign(RA_sd = np.nan)
    #df['RA_sd'] = np.where((df['SD5']<df['RA']), df['SD5'],
                     #      np.where(((df['RA'].fillna(0) != 0) & (df['RA_sd'].isna()) & (df['SD5'].fillna(0)!= 0)), df['RA'], df['RA_sd'])) 
    #// identificamos deuda que no se financiarà
    #df['t5'] = df.groupby('PLIEGO')['RA_sd'].transform('sum')
    #df['SD6'] = df['SD5'] - df['t5']
    
    # Eliminar columnas innecesarias
    df.drop(columns=[ "t2", "SD4"], inplace=True)
    
    variables = ['C_FONCOR_sd']
    
    for var in variables: 
        df[var] = df[var].fillna(0)
    # Generando ratios de la propuesta para financiar todo el SD (Tesoro) con RD: pago realizado vs recurso total
    df['ratio_t_foncor'] = df['C_FONCOR_sd']/df['C_FONCOR']
   # df['ratio_t_or'] = df['OR_sd']/df['OR']
    #df['ratio_t_ra'] = df['RA_sd']/df['RA']
    #Ya no se promedio con el ratio de ejecución promedio dado que se decidió priorizar el pago de la deuda 
    #(Uso los ratios totales de la propuesta de DGETP no los finales promedio.)
    # Calcular las variables ajustadas
    df['C_FONCOR_a'] = round(df['C_FONCOR'] * df['ratio_t_foncor'])
    df['C_FONCOR_a'].fillna(0, inplace=True)

  #  df['OR_a'] = round(df['OR'] * df['ratio_t_or'])
   # df['OR_a'].fillna(0, inplace=True)

   # df['RA_a'] = round(df['RA'] * df['ratio_t_ra'])
   # df['RA_a'].fillna(0, inplace=True)

    # Calcular SD_ax
    df[f"SD_a{year}"] = df[["C_FONCOR_a"]].sum(axis=1) 

    # Eliminar las columnas especificadas
    columns_to_drop = ['C_FONCOR_sd', 'ratio_t_foncor']
    df.drop(columns=columns_to_drop, inplace=True)
    
    ########Guardamos el PIA reducido (menos los recursos para la GG 2.8)
    
    # Reemplazar valores en f"PIA_BASE_{year}"
    df[f"PIA_BASE_{year}"] = df[f"PIA_BASE_{year}"] - df[f"SD_a{year}"]

    # Reemplazar valores en OR, RA y C_SC_RM
   # df['OR'] = df['OR'] - df['OR_a']
   # df['RA'] = df['RA'] - df['RA_a']
    df['C_FONCOR'] = df['C_FONCOR'] - df['C_FONCOR_a']

    # Renombrar columnas
   # df.rename(columns={'OR': 'Otros_Rec', 'RA': 'R_A'}, inplace=True)

    # Eliminar columnas especificadas
    columns_to_drop = [f"SD_a{year}", 'C_FONCOR_a', f"SD_{year}_T"]
    df.drop(columns=columns_to_drop, inplace=True)
    df['NIVEL_GOBIERNO'] = "2. GOBIERNOS REGIONALES"
    df['SECTOR'] = "99. GOBIERNOS REGIONALES"
    df['FUENTE'] = "5. RECURSOS DETERMINADOS"
    df['RUBRO'] = "15. FONDO DE COMPENSACION REGIONAL - FONCOR"
    
    return df

In [220]:
PIA_2025=PIASBF1(PIA_2025_bd24,2025)
PIA_2025.info()
PIA_2025.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AÑO             23 non-null     int64  
 1   PLIEGO          23 non-null     object 
 2   C_FONCOR        23 non-null     float64
 3   Res_Direct      23 non-null     int64  
 4   SB_2025         23 non-null     float64
 5   PMG_2025_gg3_1  23 non-null     float64
 6   PMG_2025_gg6_2  23 non-null     float64
 7   PIA_BASE_2025   23 non-null     float64
 8   NIVEL_GOBIERNO  23 non-null     object 
 9   SECTOR          23 non-null     object 
 10  FUENTE          23 non-null     object 
 11  RUBRO           23 non-null     object 
dtypes: float64(5), int64(2), object(5)
memory usage: 2.3+ KB


,AÑO,PLIEGO,C_FONCOR,Res_Direct,SB_2025,PMG_2025_gg3_1,PMG_2025_gg6_2,PIA_BASE_2025,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO
0,2025,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,233549217.0,233549217,0.0,0.0,0.0,233549217.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
1,2025,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,169753667.0,169753667,15000000.0,0.0,0.0,184753667.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
2,2025,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,97313757.0,97313757,450000.0,0.0,0.0,97763757.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
3,2025,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,352580607.0,352580607,41000000.0,0.0,0.0,393580607.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
4,2025,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,236318662.0,236318662,1000000.0,0.0,0.0,237318662.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
5,2025,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,580202373.0,580202373,8135129.0,8135129.0,0.0,588337502.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
6,2025,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,113793970.0,113793970,100000.0,0.0,0.0,113893970.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
7,2025,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,375036067.0,375036067,57629700.0,7629700.0,48000000.0,432665767.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
8,2025,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,214575762.0,214575762,99441011.0,0.0,0.0,314016773.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
9,2025,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,576784786.0,576784786,0.0,0.0,0.0,576784786.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR


In [221]:
PIA_2026=PIASBF1(PIA_2026_bd24,2026)
PIA_2026.info()
PIA_2026.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AÑO             23 non-null     int64  
 1   PLIEGO          23 non-null     object 
 2   C_FONCOR        23 non-null     float64
 3   Res_Direct      23 non-null     int64  
 4   SB_2026         23 non-null     float64
 5   PMG_2026_gg3_1  23 non-null     float64
 6   PMG_2026_gg6_2  23 non-null     float64
 7   PIA_BASE_2026   23 non-null     float64
 8   NIVEL_GOBIERNO  23 non-null     object 
 9   SECTOR          23 non-null     object 
 10  FUENTE          23 non-null     object 
 11  RUBRO           23 non-null     object 
dtypes: float64(5), int64(2), object(5)
memory usage: 2.3+ KB


,AÑO,PLIEGO,C_FONCOR,Res_Direct,SB_2026,PMG_2026_gg3_1,PMG_2026_gg6_2,PIA_BASE_2026,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO
0,2026,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,239184885.0,239184885,0.0,0.0,0.0,239184885.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
1,2026,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,180776045.0,180776045,15000000.0,0.0,0.0,195776045.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
2,2026,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,101018446.0,101018446,539700.0,0.0,0.0,101558146.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
3,2026,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,365157492.0,365157492,41000000.0,0.0,0.0,406157492.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
4,2026,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,243050883.0,243050883,1100000.0,0.0,0.0,244150883.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
5,2026,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,594773377.0,594773377,8343833.0,8343833.0,0.0,603117210.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
6,2026,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,117408642.0,117408642,100000.0,0.0,0.0,117508642.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
7,2026,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,383742958.0,383742958,57629700.0,7629700.0,48000000.0,441372658.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
8,2026,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,223197246.0,223197246,99441011.0,0.0,0.0,322638257.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
9,2026,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,590958072.0,590958072,0.0,0.0,0.0,590958072.0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR


In [222]:
PIA_2027=PIASBF1(PIA_2027_bd24,2027)
PIA_2027.info()
PIA_2027.head(28)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AÑO             23 non-null     int64  
 1   PLIEGO          23 non-null     object 
 2   C_FONCOR        23 non-null     float64
 3   Res_Direct      23 non-null     int64  
 4   SB_2027         23 non-null     float64
 5   PMG_2027_gg3_1  23 non-null     float64
 6   PMG_2027_gg6_2  23 non-null     float64
 7   PIA_BASE_2027   23 non-null     float64
 8   NIVEL_GOBIERNO  23 non-null     object 
 9   SECTOR          23 non-null     object 
 10  FUENTE          23 non-null     object 
 11  RUBRO           23 non-null     object 
dtypes: float64(5), int64(2), object(5)
memory usage: 2.3+ KB


,AÑO,PLIEGO,C_FONCOR,Res_Direct,SB_2027,PMG_2027_gg3_1,PMG_2027_gg6_2,PIA_BASE_2027,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO
0,2027,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,247521649.0,247521649,0.0,0.0,0.0,2.475216e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
1,2027,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,197139809.0,197139809,15000000.0,0.0,0.0,2.121398e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
2,2027,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,106510198.0,106510198,550000.0,0.0,0.0,1.070602e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
3,2027,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,383796675.0,383796675,41000000.0,0.0,0.0,4.247967e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
4,2027,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,253018460.0,253018460,1200000.0,0.0,0.0,2.542185e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
5,2027,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,616332873.0,616332873,8748253.0,8748253.0,0.0,6.250811e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
6,2027,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,122763115.0,122763115,100000.0,0.0,0.0,1.228631e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
7,2027,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,396620038.0,396620038,57629700.0,7629700.0,48000000.0,4.542497e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
8,2027,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,235979979.0,235979979,99489678.0,0.0,0.0,3.354697e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR
9,2027,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,611926567.0,611926567,0.0,0.0,0.0,6.119266e+08,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR


In [223]:
PIA_2025_1 = PIA_2025.copy()
PIA_2026_1 = PIA_2026.copy()
PIA_2027_1 = PIA_2027.copy()
SDT_2025_1 = SDT_2025.copy()
SDT_2026_1 = SDT_2026.copy()
SDT_2027_1 = SDT_2027.copy()
PIA_2025_base241= PIA_2025_base24.copy()
PIA_2026_base241=PIA_2026_base24.copy()
PIA_2027_base241=PIA_2027_base24.copy()

### FINANCIO OTRAS GGs (Cruce con ratios para la distribución con las otras GG de gasto - GENERANDO PROPUESTA CON RATIOS PROMEDIOS )

In [224]:
#Cruce con ratios para la distribución con las otras GG de gasto* (GENERANDO PROPUESTA CON RATIOS PROMEDIOS)

def PropyRatios(df, df1, df2, year):
    
    ratios_sb_c1= ratios_sb_c.copy()
    ratios_sb_c1.drop(columns=['mean_ratio_FONCOR', 'comp'], inplace=True)

    # aplicatr merge bases PIA_202* con ratios_sb_c1
    df = pd.merge(ratios_sb_c1, df[["PLIEGO", 'C_FONCOR', f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2", f"PIA_BASE_{year}"]], on="PLIEGO", how="outer", validate = 'm:1')
    postmerge(df)

    #  Ordenar columnas 
    df = df.reindex(columns=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA', 'ratio_f_c',
                                               'C_FONCOR',  f"SB_{year}", f"PMG_{year}_gg3_1", f"PMG_{year}_gg6_2", f"PIA_BASE_{year}"])

    # Calcular nuevas columnas 
    df['PR_C_FONCOR'] = round(df['ratio_f_c'] * df['C_FONCOR'])
    #df['PR_OR'] = df['Otros_Rec'].where(df['GENERICA'].str[0] == '6')
    #df['PR_RA'] = round(df['ratio_f_ra'] * df['R_A'])
    df[f"PR_SB_{year}"] = np.nan
    df[f"PR_SB_{year}"] = np.where((df[f"PR_SB_{year}"].isna())&(df[f"PMG_{year}_gg3_1"].fillna(0)>1)&(df['GENERICA'].str[0] == '3'), round(df[f"PMG_{year}_gg3_1"]), df[f"PR_SB_{year}"])
    df[f"PR_SB_{year}"] = np.where((df[f"PR_SB_{year}"].isna())&(df[f"PMG_{year}_gg3_1"].fillna(0)>1)&(df['GENERICA'].str[0] == '6'), round(df[f"SB_{year}"] - df[f"PMG_{year}_gg3_1"]), df[f"PR_SB_{year}"])
    df[f"PR_SB_{year}"] = np.where((df[f"PR_SB_{year}"].isna())&(df[f"PMG_{year}_gg3_1"].fillna(0)<=0)&(df['GENERICA'].str[0] == '6'), round(df[f"SB_{year}"]), df[f"PR_SB_{year}"])
    
   # df[f"PR_SB_{year}"] = round(df['ratio_sb_fed'] * df[f"SB_{year}"]).where(df['GENERICA'].str[0] == '3')
  #  df.loc[df['GENERICA'].str[0] == '6', f"PR_SB_{year}"] = round((1 - df['ratio_sb_fed']) * df[ f"SB_{year}"])

    df['PROP_RATIOS'] = df[['PR_C_FONCOR',  f"PR_SB_{year}"]].sum(axis=1)
    df['dif'] = df[f"PIA_BASE_{year}"] - df['PROP_RATIOS']
    df['tot_p'] = df[['PR_C_FONCOR', f"SB_{year}"]].sum(axis=1)
    df['dif2'] = df[f"PIA_BASE_{year}"] - df['tot_p']

    df.drop(columns=['tot_p', 'dif2'], inplace=True)
    df.rename(columns=lambda x: x.upper(), inplace=True)

    # Concatenar data( // incluimos servicio de la deuda)
    df = pd.concat([df, df1], ignore_index=True)

    # Reemplazar valores missing
    df['PROP_RATIOS'].fillna(0, inplace=True)
    df['MONTOSD_RD'].fillna(0, inplace=True)

    df['T'] = df[['PROP_RATIOS', 'MONTOSD_RD']].sum(axis=1)

    # Eliminar filas que contengan T == 0
    df = df[df['T'].fillna(0) != 0]
    
    #*Dando formato a propuesta final
    # Dando formato a propuesta final
    df['PROP_RATIOS'].fillna(0, inplace=True)
    df['MONTOSD_RD'].fillna(0, inplace=True)
    df['MONTOF_RD'] = df[['PROP_RATIOS', 'MONTOSD_RD']].sum(axis=1)
    df['NIVEL_GOBIERNO'].replace({'GOBIERNOS REGIONALES': '2. GOBIERNOS REGIONALES'}, inplace=True)
    
    df['CATEGORIA_GTO'] = np.where(df['CATEGORIA_GTO'].str.contains('5'),'5. GASTOS CORRIENTES',
                                     np.where(df['CATEGORIA_GTO'].str.contains('6'), '6. GASTOS DE CAPITAL', df['CATEGORIA_GTO']))
    
    # Ajustes de redondeo
    df['PROP'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO'])['MONTOF_RD'].transform('sum')
    df.drop(columns=[f"PIA_BASE_{year}"], inplace=True)

    df = pd.merge(df, df2, on='PLIEGO', how="outer", validate = 'm:1')
    postmerge(df)
    df['difb'] = df[f"PIA_BASE_{year}"] - df['PROP']
    df['MONTOF_RD'].fillna(0, inplace=True)
    df['difb'].fillna(0, inplace=True)
    df['MONTOF_RD'] = np.where(((df['difb'] < 0) & (df['GENERICA'].str[0] == '3')), df[['MONTOF_RD', 'difb']].sum(axis=1),
                               np.where((df['difb'] > 0) & (df['GENERICA'].str[0] == '6'), df[['MONTOF_RD', 'difb']].sum(axis=1), df['MONTOF_RD']))
    df.drop(columns=['difb'], inplace=True)
    #*c. Ajustes de distribución por marco legal (c1. REASIGNANDO EL MONTO DE LA GG 2.4)
    df['Monto_GG4_CO'] = np.where(((df['GENERICA'].str[0] == '4') & (df['CATEGORIA_GTO'].str[0] == '5')), df['MONTOF_RD'], np.nan)    
    df['Monto_GG4_CA'] = np.where(((df['GENERICA'].str[0] == '4') & (df['CATEGORIA_GTO'].str[0] == '6')), df['MONTOF_RD'], np.nan)    
    # Calcular sumas por grupos
    df['PLIEGO_GG24_CO'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA'])['Monto_GG4_CO'].transform('sum')
    df['PLIEGO_GG24_CA'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA'])['Monto_GG4_CA'].transform('sum')

    # Reemplazar valores nulos con 0
    df['PLIEGO_GG24_CO'].fillna(0, inplace=True)
    df['PLIEGO_GG24_CA'].fillna(0, inplace=True)

    # Actualizar MONTOF_RD según condiciones
    df['MONTOF_RD'] = np.where(df['GENERICA'].str[0] == '3', df[['MONTOF_RD', 'PLIEGO_GG24_CO']].sum(axis=1), df['MONTOF_RD'])
    df['MONTOF_RD'] = np.where(df['GENERICA'].str[0] == '6', df[['MONTOF_RD', 'PLIEGO_GG24_CA']].sum(axis=1), df['MONTOF_RD'])
    df['MONTOF_RD'] = np.where(df['GENERICA'].str[0] == '4', df['MONTOF_RD'], df['MONTOF_RD'])
    
    #*c2. REASIGNANDO EL MONTO DE LA GG 2.5 (CALLAO Y LORETO ) - Loreto siempre fue la única en tener Capital - por eso se hace ajuste para Callao
    df['Monto_GG5_CO'] = np.where(((df['GENERICA'].str[0] == '5') & (df['CATEGORIA_GTO'].str[0] == '5')), df['MONTOF_RD'], np.nan)
    df['Monto_GG5_CA'] = np.where(((df['GENERICA'].str[0] == '5') & (df['CATEGORIA_GTO'].str[0] == '6')), df['MONTOF_RD'], np.nan)   
    
    # Calcular sumas por grupos
    df['PLIEGO_GG25_CO'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA' ])['Monto_GG5_CO'].transform('sum')
    df['PLIEGO_GG25_CA'] = df.groupby(['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA' ])['Monto_GG5_CA'].transform('sum')
    
    # Reemplazar valores nulos con 0
    df['PLIEGO_GG25_CO'].fillna(0, inplace=True)
    df['PLIEGO_GG25_CA'].fillna(0, inplace=True)

    
    # Reemplazar MONTOF_RD según la primera condición
    df['MONTOF_RD'] = np.where((df['GENERICA'].str.contains('3') & ~(df['PLIEGO'].str.contains('464'))), df[['MONTOF_RD', 'PLIEGO_GG25_CO']].sum(axis=1),
                               np.where((df['GENERICA'].str.contains('5') & df['CATEGORIA_GTO'].str.contains('5') & ~(df['PLIEGO'].str.contains('464'))), 0, df['MONTOF_RD']))

    # Reemplazar GENERICA según la tercera condición
    df.loc[(df['GENERICA'] == '7. ADQUISICION DE ACTIVOS FINANCIEROS') & (df['PLIEGO'] == '453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO'), 'GENERICA'] = '6. ADQUISICION DE ACTIVOS NO FINANCIEROS'
    #* c3. VERIFICANDO LA PARTICIPACION DEL GASTO CORRIENTE ( (20% DEL CANON, SOBRECANON Y RM PARA MANTENIMIENTO))
    
    # Verificando la participación del gasto corriente
    df['Totfoncor'] = df.groupby('PLIEGO')['PR_C_FONCOR'].transform('sum')
    df['ratio_foncor'] = df['PR_C_FONCOR'] / df['Totfoncor']
    df['EXCESO_GG3'] = np.where((df['ratio_foncor'] > 0.2) & df['GENERICA'].str.contains('3'), np.round(df['ratio_foncor']- 0.2, 4), np.nan)
    df['MOD'] = np.round(df['EXCESO_GG3'] * df[f"PIA_BASE_{year}"])    
    df['TotMOD'] = df.groupby('PLIEGO')['MOD'].transform('sum')
    df['MONTOF_RD'].fillna(0, inplace=True)
    df['TotMOD'].fillna(0, inplace=True)
    df['MONTOF_RD'] = np.where(df['GENERICA'].str.contains('6') & (~df['PLIEGO'].str.contains('CALLAO')),df[['MONTOF_RD', 'TotMOD']].sum(axis=1),
                               np.where(df['GENERICA'].str.contains('3') & (~df['PLIEGO'].str.contains('CALLAO')), df['MONTOF_RD'] - df['TotMOD'], df['MONTOF_RD']))
    df['ratio_foncor2'] = df['PR_C_FONCOR'] / df['Totfoncor']
    df['EXCESO_GG3_2'] = np.where((df['ratio_foncor2'] > 0.2) & df['GENERICA'].str.contains('3'), np.round(df['ratio_foncor2'] - 0.2, 4), np.nan)
    
    columnas = list(df.columns)
    df = df[["NIVEL_GOBIERNO"] + columnas[:31] + columnas[32:]]
    df.drop(columns=['NIVEL_GOBIERNO'], inplace=True)
    df['NIVEL_GOBIERNO'] = '2. GOBIERNOS REGIONALES'
    df['RUBRO'] = '15. FONDO DE COMPENSACION REGIONAL - FONCOR'
    return df

In [225]:
PropyRatios25=PropyRatios(PIA_2025_1,SDT_2025_1, PIA_2025_base241, 2025)
PropyRatios25.info()
PropyRatios25.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          57 non-null     object 
 1   CATEGORIA_GTO   57 non-null     object 
 2   GENERICA        57 non-null     object 
 3   RATIO_F_C       57 non-null     float64
 4   C_FONCOR        57 non-null     float64
 5   PMG_2025_GG3_1  57 non-null     float64
 6   PMG_2025_GG6_2  57 non-null     float64
 7   PR_C_FONCOR     57 non-null     float64
 8   PR_SB_2025      26 non-null     float64
 9   PROP_RATIOS     57 non-null     float64
 10  DIF             57 non-null     float64
 11  MONTOSD_RD      57 non-null     float64
 12  T               57 non-null     float64
 13  MONTOF_RD       57 non-null     float64
 14  PROP            57 non-null     float64
 15  Foncor          57 non-null     int64  
 16  Res_Direct      57 non-null     int64  
 17  PMG_2025_gg3_1  57 non-null     float

,PLIEGO,CATEGORIA_GTO,GENERICA,RATIO_F_C,C_FONCOR,PMG_2025_GG3_1,PMG_2025_GG6_2,PR_C_FONCOR,PR_SB_2025,PROP_RATIOS,...,PLIEGO_GG25_CO,PLIEGO_GG25_CA,Totfoncor,ratio_foncor,EXCESO_GG3,MOD,TotMOD,ratio_foncor2,EXCESO_GG3_2,NIVEL_GOBIERNO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.003239,233549217.0,0.0,0.0,756405.0,NaN,756405.0,...,0.0,0.0,233549217.0,0.003239,NaN,NaN,0.0,0.003239,NaN,2. GOBIERNOS REGIONALES
1,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.996761,233549217.0,0.0,0.0,232792812.0,0.0,232792812.0,...,0.0,0.0,233549217.0,0.996761,NaN,NaN,0.0,0.996761,NaN,2. GOBIERNOS REGIONALES
2,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.000119,169753667.0,0.0,0.0,20255.0,NaN,20255.0,...,0.0,0.0,169753667.0,0.000119,NaN,NaN,0.0,0.000119,NaN,2. GOBIERNOS REGIONALES
3,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.999881,169753667.0,0.0,0.0,169733412.0,15000000.0,184733412.0,...,0.0,0.0,169753667.0,0.999881,NaN,NaN,0.0,0.999881,NaN,2. GOBIERNOS REGIONALES
4,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.001745,97313757.0,0.0,0.0,169831.0,NaN,169831.0,...,0.0,0.0,97313757.0,0.001745,NaN,NaN,0.0,0.001745,NaN,2. GOBIERNOS REGIONALES
5,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.000899,97313757.0,0.0,0.0,87461.0,NaN,87461.0,...,0.0,0.0,97313757.0,0.000899,NaN,NaN,0.0,0.000899,NaN,2. GOBIERNOS REGIONALES
6,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.997356,97313757.0,0.0,0.0,97056465.0,450000.0,97506465.0,...,0.0,0.0,97313757.0,0.997356,NaN,NaN,0.0,0.997356,NaN,2. GOBIERNOS REGIONALES
7,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.051363,352580607.0,0.0,0.0,18109527.0,NaN,18109527.0,...,0.0,0.0,352580608.0,0.051363,NaN,NaN,0.0,0.051363,NaN,2. GOBIERNOS REGIONALES
8,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.090296,352580607.0,0.0,0.0,31836751.0,NaN,31836751.0,...,0.0,0.0,352580608.0,0.090296,NaN,NaN,0.0,0.090296,NaN,2. GOBIERNOS REGIONALES
9,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.858341,352580607.0,0.0,0.0,302634330.0,41000000.0,343634330.0,...,0.0,0.0,352580608.0,0.858341,NaN,NaN,0.0,0.858341,NaN,2. GOBIERNOS REGIONALES


In [226]:
PropyRatios26=PropyRatios(PIA_2026_1,SDT_2026_1, PIA_2026_base241, 2026)
PropyRatios26.info()
PropyRatios26.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          57 non-null     object 
 1   CATEGORIA_GTO   57 non-null     object 
 2   GENERICA        57 non-null     object 
 3   RATIO_F_C       57 non-null     float64
 4   C_FONCOR        57 non-null     float64
 5   PMG_2026_GG3_1  57 non-null     float64
 6   PMG_2026_GG6_2  57 non-null     float64
 7   PR_C_FONCOR     57 non-null     float64
 8   PR_SB_2026      26 non-null     float64
 9   PROP_RATIOS     57 non-null     float64
 10  DIF             57 non-null     float64
 11  MONTOSD_RD      57 non-null     float64
 12  T               57 non-null     float64
 13  MONTOF_RD       57 non-null     float64
 14  PROP            57 non-null     float64
 15  Foncor          57 non-null     int64  
 16  Res_Direct      57 non-null     int64  
 17  PMG_2026_gg3_1  57 non-null     float

,PLIEGO,CATEGORIA_GTO,GENERICA,RATIO_F_C,C_FONCOR,PMG_2026_GG3_1,PMG_2026_GG6_2,PR_C_FONCOR,PR_SB_2026,PROP_RATIOS,...,PLIEGO_GG25_CO,PLIEGO_GG25_CA,Totfoncor,ratio_foncor,EXCESO_GG3,MOD,TotMOD,ratio_foncor2,EXCESO_GG3_2,NIVEL_GOBIERNO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.003239,239184885.0,0.0,0.0,774658.0,NaN,774658.0,...,0.0,0.0,239184885.0,0.003239,NaN,NaN,0.0,0.003239,NaN,2. GOBIERNOS REGIONALES
1,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.996761,239184885.0,0.0,0.0,238410227.0,0.0,238410227.0,...,0.0,0.0,239184885.0,0.996761,NaN,NaN,0.0,0.996761,NaN,2. GOBIERNOS REGIONALES
2,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.000119,180776045.0,0.0,0.0,21570.0,NaN,21570.0,...,0.0,0.0,180776045.0,0.000119,NaN,NaN,0.0,0.000119,NaN,2. GOBIERNOS REGIONALES
3,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.999881,180776045.0,0.0,0.0,180754475.0,15000000.0,195754475.0,...,0.0,0.0,180776045.0,0.999881,NaN,NaN,0.0,0.999881,NaN,2. GOBIERNOS REGIONALES
4,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.001745,101018446.0,0.0,0.0,176296.0,NaN,176296.0,...,0.0,0.0,101018446.0,0.001745,NaN,NaN,0.0,0.001745,NaN,2. GOBIERNOS REGIONALES
5,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.000899,101018446.0,0.0,0.0,90791.0,NaN,90791.0,...,0.0,0.0,101018446.0,0.000899,NaN,NaN,0.0,0.000899,NaN,2. GOBIERNOS REGIONALES
6,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.997356,101018446.0,0.0,0.0,100751359.0,539700.0,101291059.0,...,0.0,0.0,101018446.0,0.997356,NaN,NaN,0.0,0.997356,NaN,2. GOBIERNOS REGIONALES
7,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.051363,365157492.0,0.0,0.0,18755511.0,NaN,18755511.0,...,0.0,0.0,365157492.0,0.051363,NaN,NaN,0.0,0.051363,NaN,2. GOBIERNOS REGIONALES
8,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.090296,365157492.0,0.0,0.0,32972398.0,NaN,32972398.0,...,0.0,0.0,365157492.0,0.090296,NaN,NaN,0.0,0.090296,NaN,2. GOBIERNOS REGIONALES
9,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.858341,365157492.0,0.0,0.0,313429583.0,41000000.0,354429583.0,...,0.0,0.0,365157492.0,0.858341,NaN,NaN,0.0,0.858341,NaN,2. GOBIERNOS REGIONALES


In [227]:
PropyRatios27=PropyRatios(PIA_2027_1,SDT_2027_1, PIA_2027_base241, 2027)
PropyRatios27.info()
PropyRatios27.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLIEGO          57 non-null     object 
 1   CATEGORIA_GTO   57 non-null     object 
 2   GENERICA        57 non-null     object 
 3   RATIO_F_C       57 non-null     float64
 4   C_FONCOR        57 non-null     float64
 5   PMG_2027_GG3_1  57 non-null     float64
 6   PMG_2027_GG6_2  57 non-null     float64
 7   PR_C_FONCOR     57 non-null     float64
 8   PR_SB_2027      26 non-null     float64
 9   PROP_RATIOS     57 non-null     float64
 10  DIF             57 non-null     float64
 11  MONTOSD_RD      57 non-null     float64
 12  T               57 non-null     float64
 13  MONTOF_RD       57 non-null     float64
 14  PROP            57 non-null     float64
 15  Foncor          57 non-null     int64  
 16  Res_Direct      57 non-null     int64  
 17  PMG_2027_gg3_1  57 non-null     float

,PLIEGO,CATEGORIA_GTO,GENERICA,RATIO_F_C,C_FONCOR,PMG_2027_GG3_1,PMG_2027_GG6_2,PR_C_FONCOR,PR_SB_2027,PROP_RATIOS,...,PLIEGO_GG25_CO,PLIEGO_GG25_CA,Totfoncor,ratio_foncor,EXCESO_GG3,MOD,TotMOD,ratio_foncor2,EXCESO_GG3_2,NIVEL_GOBIERNO
0,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.003239,247521649.0,0.0,0.0,801658.0,NaN,801658.0,...,0.0,0.0,247521649.0,0.003239,NaN,NaN,0.0,0.003239,NaN,2. GOBIERNOS REGIONALES
1,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.996761,247521649.0,0.0,0.0,246719991.0,0.0,246719991.0,...,0.0,0.0,247521649.0,0.996761,NaN,NaN,0.0,0.996761,NaN,2. GOBIERNOS REGIONALES
2,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.000119,197139809.0,0.0,0.0,23523.0,NaN,23523.0,...,0.0,0.0,197139809.0,0.000119,NaN,NaN,0.0,0.000119,NaN,2. GOBIERNOS REGIONALES
3,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.999881,197139809.0,0.0,0.0,197116286.0,15000000.0,212116286.0,...,0.0,0.0,197139809.0,0.999881,NaN,NaN,0.0,0.999881,NaN,2. GOBIERNOS REGIONALES
4,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.001745,106510198.0,0.0,0.0,185880.0,NaN,185880.0,...,0.0,0.0,106510198.0,0.001745,NaN,NaN,0.0,0.001745,NaN,2. GOBIERNOS REGIONALES
5,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.000899,106510198.0,0.0,0.0,95727.0,NaN,95727.0,...,0.0,0.0,106510198.0,0.000899,NaN,NaN,0.0,0.000899,NaN,2. GOBIERNOS REGIONALES
6,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.997356,106510198.0,0.0,0.0,106228591.0,550000.0,106778591.0,...,0.0,0.0,106510198.0,0.997356,NaN,NaN,0.0,0.997356,NaN,2. GOBIERNOS REGIONALES
7,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.051363,383796675.0,0.0,0.0,19712871.0,NaN,19712871.0,...,0.0,0.0,383796675.0,0.051363,NaN,NaN,0.0,0.051363,NaN,2. GOBIERNOS REGIONALES
8,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.090296,383796675.0,0.0,0.0,34655449.0,NaN,34655449.0,...,0.0,0.0,383796675.0,0.090296,NaN,NaN,0.0,0.090296,NaN,2. GOBIERNOS REGIONALES
9,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,0.858341,383796675.0,0.0,0.0,329428355.0,41000000.0,370428355.0,...,0.0,0.0,383796675.0,0.858341,NaN,NaN,0.0,0.858341,NaN,2. GOBIERNOS REGIONALES


In [228]:
PropyRatios25_1 = PropyRatios25.copy()
PropyRatios26_1 = PropyRatios26.copy()
PropyRatios27_1 = PropyRatios27.copy()

In [229]:
def MONTOF_RD(data, year):
    
    #*Formato Pliego por tipo de recurso
    data.rename(columns=lambda x: x.upper(), inplace=True)
    # Calcular MONTOI_RD
    data['MONTOI_RD'] = data[['PROP_RATIOS', 'MONTOSD_RD']].sum(axis=1) 

    # Calcular AJUSTES
    data['AJUSTES'] = data['MONTOF_RD'] - data['MONTOI_RD']

    # Agrupar y colapsar
    data = data.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA']).agg({'PR_C_FONCOR': 'sum', f"PR_SB_{year}": 'sum',
                                                                                                                     'AJUSTES': 'sum', 'MONTOI_RD': 'sum',
                                                                                                                     'MONTOF_RD': 'sum'}).reset_index()

    # Reemplazar PR_C_SC_RM según la condición
    data['PR_C_FONCOR'] = data[['PR_C_FONCOR', 'AJUSTES']].sum(axis=1) 

    # Eliminar filas donde MONTOI_RD es 0
    data = data[data['MONTOI_RD']. fillna(0) != 0]
    
    #data['PR_RA'].fillna(0, inplace=True)
    data['AJUSTES'].fillna(0, inplace=True)
    
    # Reemplazar PR_RA según la condición
   # data['PR_RA'] = np.where(data['PR_C_SC_RM'] < 0, data[['PR_RA', 'AJUSTES']].sum(axis=1), data['PR_RA'])

    # Reemplazar valores negativos de PR_C_SC_RM con 0
    data['PR_C_FONCOR'] = np.where(data['PR_C_FONCOR'] < 0, 0, data['PR_C_FONCOR'])

    # Agregar columna AÑO
    data['AÑO'] = year

    # Ordenar columnas
    data = data[['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'AÑO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA', 'MONTOI_RD', 'AJUSTES', 'MONTOF_RD', 'PR_C_FONCOR', f"PR_SB_{year}"]]

    # Renombrar columnas
    data.rename(columns={'PR_C_FONCOR': f"PR_C_FONCOR{year}", 'MONTOF_RD': f'PMG_{year}'}, inplace=True)

    return data

In [230]:
MONTOF_RD_25=MONTOF_RD(PropyRatios25_1, 2025)
MONTOF_RD_25.info()
MONTOF_RD_25.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NIVEL_GOBIERNO   57 non-null     object 
 1   SECTOR           57 non-null     object 
 2   FUENTE           57 non-null     object 
 3   RUBRO            57 non-null     object 
 4   AÑO              57 non-null     int64  
 5   PLIEGO           57 non-null     object 
 6   CATEGORIA_GTO    57 non-null     object 
 7   GENERICA         57 non-null     object 
 8   MONTOI_RD        57 non-null     float64
 9   AJUSTES          57 non-null     float64
 10  PMG_2025         57 non-null     float64
 11  PR_C_FONCOR2025  57 non-null     float64
 12  PR_SB_2025       57 non-null     float64
dtypes: float64(5), int64(1), object(7)
memory usage: 5.9+ KB


,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO,AÑO,PLIEGO,CATEGORIA_GTO,GENERICA,MONTOI_RD,AJUSTES,PMG_2025,PR_C_FONCOR2025,PR_SB_2025
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,756405.0,0.0,756405.0,756405.0,0.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,232792812.0,0.0,232792812.0,232792812.0,0.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,20255.0,0.0,20255.0,20255.0,0.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,184733412.0,0.0,184733412.0,169733412.0,15000000.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,169831.0,0.0,169831.0,169831.0,0.0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,87461.0,0.0,87461.0,87461.0,0.0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,97506465.0,0.0,97506465.0,97056465.0,450000.0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,18109527.0,-1.0,18109526.0,18109526.0,0.0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,31836751.0,0.0,31836751.0,31836751.0,0.0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2025,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,343634330.0,0.0,343634330.0,302634330.0,41000000.0


In [231]:
MONTOF_RD_26=MONTOF_RD(PropyRatios26_1, 2026)
MONTOF_RD_26.info()
MONTOF_RD_26.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NIVEL_GOBIERNO   57 non-null     object 
 1   SECTOR           57 non-null     object 
 2   FUENTE           57 non-null     object 
 3   RUBRO            57 non-null     object 
 4   AÑO              57 non-null     int64  
 5   PLIEGO           57 non-null     object 
 6   CATEGORIA_GTO    57 non-null     object 
 7   GENERICA         57 non-null     object 
 8   MONTOI_RD        57 non-null     float64
 9   AJUSTES          57 non-null     float64
 10  PMG_2026         57 non-null     float64
 11  PR_C_FONCOR2026  57 non-null     float64
 12  PR_SB_2026       57 non-null     float64
dtypes: float64(5), int64(1), object(7)
memory usage: 5.9+ KB


,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO,AÑO,PLIEGO,CATEGORIA_GTO,GENERICA,MONTOI_RD,AJUSTES,PMG_2026,PR_C_FONCOR2026,PR_SB_2026
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,774658.0,0.0,774658.0,774658.0,0.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,238410227.0,0.0,238410227.0,238410227.0,0.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,21570.0,0.0,21570.0,21570.0,0.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,195754475.0,0.0,195754475.0,180754475.0,15000000.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,176296.0,0.0,176296.0,176296.0,0.0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,90791.0,0.0,90791.0,90791.0,0.0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,101291059.0,0.0,101291059.0,100751359.0,539700.0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,18755511.0,0.0,18755511.0,18755511.0,0.0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,32972398.0,0.0,32972398.0,32972398.0,0.0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2026,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,354429583.0,0.0,354429583.0,313429583.0,41000000.0


In [232]:
MONTOF_RD_27=MONTOF_RD(PropyRatios27_1, 2027)
MONTOF_RD_27.info()
MONTOF_RD_27.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NIVEL_GOBIERNO   57 non-null     object 
 1   SECTOR           57 non-null     object 
 2   FUENTE           57 non-null     object 
 3   RUBRO            57 non-null     object 
 4   AÑO              57 non-null     int64  
 5   PLIEGO           57 non-null     object 
 6   CATEGORIA_GTO    57 non-null     object 
 7   GENERICA         57 non-null     object 
 8   MONTOI_RD        57 non-null     float64
 9   AJUSTES          57 non-null     float64
 10  PMG_2027         57 non-null     float64
 11  PR_C_FONCOR2027  57 non-null     float64
 12  PR_SB_2027       57 non-null     float64
dtypes: float64(5), int64(1), object(7)
memory usage: 5.9+ KB


,NIVEL_GOBIERNO,SECTOR,FUENTE,RUBRO,AÑO,PLIEGO,CATEGORIA_GTO,GENERICA,MONTOI_RD,AJUSTES,PMG_2027,PR_C_FONCOR2027,PR_SB_2027
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,801658.0,0.0,801658.0,801658.0,0.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,246719991.0,0.0,246719991.0,246719991.0,0.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,23523.0,0.0,23523.0,23523.0,0.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,212116286.0,0.0,212116286.0,197116286.0,15000000.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,185880.0,0.0,185880.0,185880.0,0.0
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,95727.0,0.0,95727.0,95727.0,0.0
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,106778591.0,0.0,106778591.0,106228591.0,550000.0
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,19712871.0,0.0,19712871.0,19712871.0,0.0
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,34655449.0,0.0,34655449.0,34655449.0,0.0
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2027,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,370428355.0,0.0,370428355.0,329428355.0,41000000.0


In [233]:
MONTOF_RD_25_1=MONTOF_RD_25.copy()
MONTOF_RD_26_1=MONTOF_RD_26.copy()
MONTOF_RD_27_1=MONTOF_RD_27.copy()

In [234]:
# Realizar la primera fusión m:m
Propuesta = pd.merge(MONTOF_RD_25_1, MONTOF_RD_26_1, on=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA'], how='outer', validate='m:m')
postmerge(Propuesta)
# Realizar la segunda fusión m:m
Propuesta_RD_GG = pd.merge(Propuesta, MONTOF_RD_27_1, on=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA'], how='outer', validate='m:m')
postmerge(Propuesta_RD_GG)

# Seleccionar columnas necesarias
Propuesta_RD_GG = Propuesta_RD_GG[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA',
                                     'PMG_2025', 'PR_C_FONCOR2025', 'PR_SB_2025',
                                     'PMG_2026', 'PR_C_FONCOR2026', 'PR_SB_2026',
                                     'PMG_2027', 'PR_C_FONCOR2027', 'PR_SB_2027']]
# 1 FLUJO 2 ES SALDO
# Calcular PMG2025, PMG2026 y PMG2027
Propuesta_RD_GG['PMG20251'] = Propuesta_RD_GG[["PR_C_FONCOR2025"]].sum(axis=1)
Propuesta_RD_GG['PMG20261'] = Propuesta_RD_GG[["PR_C_FONCOR2026"]].sum(axis=1)
Propuesta_RD_GG['PMG20271'] = Propuesta_RD_GG[["PR_C_FONCOR2027"]].sum(axis=1)

# Renombrar columnas PR_SB25, PR_SB26 y PR_SB27
Propuesta_RD_GG.rename(columns={'PR_SB_2025': 'PMG20252', 'PR_SB_2026': 'PMG20262', 'PR_SB_2027': 'PMG20272'}, inplace=True)

# Seleccionar columnas requeridas
Propuesta_RD_GG = Propuesta_RD_GG[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA',
                                 'PMG20251', 'PMG20261', 'PMG20271', 'PMG20252', 'PMG20262', 'PMG20272']]

# Cambiar formato de ancho a largo (reshape)2025
Propuesta_RD_GG25 = pd.melt(Propuesta_RD_GG, id_vars=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'],
                          value_vars=['PMG20251', 'PMG20252'],
                          var_name='TIPO_GEN_ING', value_name='PMG_2025')
Propuesta_RD_GG25['TIPO_GEN_ING'] = Propuesta_RD_GG25['TIPO_GEN_ING'].str[-1]

# Cambiar formato de ancho a largo (reshape)
Propuesta_RD_GG26 = pd.melt(Propuesta_RD_GG, id_vars=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'],
                          value_vars=['PMG20261', 'PMG20262'],
                          var_name='TIPO_GEN_ING', value_name='PMG_2026')
Propuesta_RD_GG26['TIPO_GEN_ING'] = Propuesta_RD_GG26['TIPO_GEN_ING'].str[-1]

# Cambiar formato de ancho a largo (reshape)
Propuesta_RD_GG27 = pd.melt(Propuesta_RD_GG, id_vars=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'],
                          value_vars=['PMG20271', 'PMG20272'],
                          var_name='TIPO_GEN_ING', value_name='PMG_2027')
Propuesta_RD_GG27['TIPO_GEN_ING'] = Propuesta_RD_GG27['TIPO_GEN_ING'].str[-1]

Propuesta_RD_GGG = pd.merge(Propuesta_RD_GG25, Propuesta_RD_GG26, on=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA','TIPO_GEN_ING'], how='left', validate='m:m')
postmerge(Propuesta_RD_GGG)
Propuesta_RD_GGG = pd.merge(Propuesta_RD_GGG, Propuesta_RD_GG27, on=['NIVEL_GOBIERNO', 'SECTOR', 'FUENTE', 'RUBRO', 'PLIEGO', 'CATEGORIA_GTO', 'GENERICA','TIPO_GEN_ING'], how='left', validate='m:m')
postmerge(Propuesta_RD_GGG)
Propuesta_RD_GGG = Propuesta_RD_GGG[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA',
                                     'PMG_2025', 'PMG_2026', 'PMG_2027']]

Propuesta_RD_GGG['TIPO_GEN_ING'].replace({'1': '2. OTRAS GENÉRICAS', '2':'1. SALDOS DE BALANCE'}, inplace=True)
Propuesta_RD_GGG = Propuesta_RD_GGG[~((Propuesta_RD_GGG['PMG_2025'].fillna(0)==0)&(Propuesta_RD_GGG['PMG_2026'].fillna(0)==0)&(Propuesta_RD_GGG['PMG_2027'].fillna(0)==0))]
Propuesta_RD_GGG = Propuesta_RD_GGG.reset_index(drop=True)
Propuesta_RD_GGG.info()
Propuesta_RD_GGG.head(768)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOBIERNO  74 non-null     object 
 1   SECTOR          74 non-null     object 
 2   PLIEGO          74 non-null     object 
 3   FUENTE          74 non-null     object 
 4   RUBRO           74 non-null     object 
 5   TIPO_GEN_ING    74 non-null     object 
 6   CATEGORIA_GTO   74 non-null     object 
 7   GENERICA        74 non-null     object 
 8   PMG_2025        74 non-null     float64
 9   PMG_2026        74 non-null     float64
 10  PMG_2027        74 non-null     float64
dtypes: float64(3), object(8)
memory usage: 6.5+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PMG_2025,PMG_2026,PMG_2027
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,756405.0,774658.0,801658.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,232792812.0,238410227.0,246719991.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,20255.0,21570.0,23523.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,169733412.0,180754475.0,197116286.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,169831.0,176296.0,185880.0
...,...,...,...,...,...,...,...,...,...,...,...
69,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,120000000.0,120000000.0,120000000.0
70,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,458. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PUNO,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,7330000.0,7330000.0,7030000.0
71,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,459. GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,3000000.0,3500000.0,4000000.0
72,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,38931353.0,30035741.0,35147012.0


In [235]:
# Realizar correcciones de redondeo
Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] + 8, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2027'] + 3, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2025'] - 2, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2026'] + 2, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AREQUIPA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2027'] + 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYACUCHO") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2026'] + 1, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2025'] + 2, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2026'] - 1, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") & 
                                        (Propuesta_RD_GGG['PLIEGO'] == "445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJAMARCA") & 
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"), 
                                        Propuesta_RD_GGG['PMG_2027'] - 2, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 6, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] - 8, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 4, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUANCAVELICA")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] - 1, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA")&
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 16, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS")&
                                        (Propuesta_RD_GGG['PLIEGO'] == "453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] + 11, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "453. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LORETO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 18, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "455. GOBIERNO REGIONAL DEL DEPARTAMENTO DE MOQUEGUA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] + 1, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "458. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PUNO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "459. GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN MARTIN") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 4, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "459. GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN MARTIN") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] - 4, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "460. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TACNA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 2, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2026'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "460. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TACNA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2026'] + 2, Propuesta_RD_GGG['PMG_2026'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "460. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TACNA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 1, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "461. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TUMBES") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCAYALI") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 3, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCAYALI") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] + 2, Propuesta_RD_GGG['PMG_2027'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] - 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2025'] = np.where((Propuesta_RD_GGG['GENERICA'] == "6. ADQUISICION DE ACTIVOS NO FINANCIEROS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCIONAL DEL CALLAO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2025'] + 1, Propuesta_RD_GGG['PMG_2025'])

Propuesta_RD_GGG['PMG_2027'] = np.where((Propuesta_RD_GGG['GENERICA'] == "3. BIENES Y SERVICIOS") &
                                        (Propuesta_RD_GGG['PLIEGO'] == "464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTITUCIONAL DEL CALLAO") &
                                        (Propuesta_RD_GGG['TIPO_GEN_ING'] == "2. OTRAS GENÉRICAS"),
                                        Propuesta_RD_GGG['PMG_2027'] - 1, Propuesta_RD_GGG['PMG_2027'])

# Eliminar filas duplicadas en el dataframe
Propuesta_RD_GGG.drop_duplicates(inplace=True)

# Crear indicador de pliegos que superan el 20% a nivel de la GG 2.3.
Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO25']= Propuesta_RD_GGG.groupby(['PLIEGO', 'GENERICA'])['PMG_2025'].transform('sum').astype(float)
Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO26']= Propuesta_RD_GGG.groupby(['PLIEGO', 'GENERICA'])['PMG_2026'].transform('sum').astype(float)
Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO27']= Propuesta_RD_GGG.groupby(['PLIEGO', 'GENERICA'])['PMG_2027'].transform('sum').astype(float)

Propuesta_RD_GGG ['TOTAL_PLIEGO25']= Propuesta_RD_GGG.groupby(['PLIEGO'])['PMG_2025'].transform('sum').astype(float)
Propuesta_RD_GGG ['TOTAL_PLIEGO26']= Propuesta_RD_GGG.groupby(['PLIEGO'])['PMG_2026'].transform('sum').astype(float)
Propuesta_RD_GGG ['TOTAL_PLIEGO27']= Propuesta_RD_GGG.groupby(['PLIEGO'])['PMG_2027'].transform('sum').astype(float)

Propuesta_RD_GGG ['IND_GEN_PLIEGO25']= (Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO25'] / Propuesta_RD_GGG ['TOTAL_PLIEGO25'])*100
Propuesta_RD_GGG ['IND_GEN_PLIEGO26']= (Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO26'] / Propuesta_RD_GGG ['TOTAL_PLIEGO26'])*100
Propuesta_RD_GGG ['IND_GEN_PLIEGO27']= (Propuesta_RD_GGG ['TOTAL_GEN_PLIEGO27'] / Propuesta_RD_GGG ['TOTAL_PLIEGO27'])*100

Propuesta_RD_GGG ['ALERTA_SUP20%_25'] = np.where(Propuesta_RD_GGG['GENERICA'].str.contains('BIENES Y SERVICIOS')&(Propuesta_RD_GGG['IND_GEN_PLIEGO25'] > 20), 'SI', 'NO')
Propuesta_RD_GGG ['ALERTA_SUP20%_26'] = np.where(Propuesta_RD_GGG['GENERICA'].str.contains('BIENES Y SERVICIOS')&(Propuesta_RD_GGG['IND_GEN_PLIEGO26'] > 20), 'SI', 'NO')
Propuesta_RD_GGG ['ALERTA_SUP20%_27'] = np.where(Propuesta_RD_GGG['GENERICA'].str.contains('BIENES Y SERVICIOS')&(Propuesta_RD_GGG['IND_GEN_PLIEGO27'] > 20), 'SI', 'NO')
Propuesta_RD_GGG.info()
Propuesta_RD_GGG.head(139)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NIVEL_GOBIERNO      74 non-null     object 
 1   SECTOR              74 non-null     object 
 2   PLIEGO              74 non-null     object 
 3   FUENTE              74 non-null     object 
 4   RUBRO               74 non-null     object 
 5   TIPO_GEN_ING        74 non-null     object 
 6   CATEGORIA_GTO       74 non-null     object 
 7   GENERICA            74 non-null     object 
 8   PMG_2025            74 non-null     float64
 9   PMG_2026            74 non-null     float64
 10  PMG_2027            74 non-null     float64
 11  TOTAL_GEN_PLIEGO25  74 non-null     float64
 12  TOTAL_GEN_PLIEGO26  74 non-null     float64
 13  TOTAL_GEN_PLIEGO27  74 non-null     float64
 14  TOTAL_PLIEGO25      74 non-null     float64
 15  TOTAL_PLIEGO26      74 non-null     float64
 16  TOTAL_PLIE

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PMG_2025,PMG_2026,...,TOTAL_GEN_PLIEGO27,TOTAL_PLIEGO25,TOTAL_PLIEGO26,TOTAL_PLIEGO27,IND_GEN_PLIEGO25,IND_GEN_PLIEGO26,IND_GEN_PLIEGO27,ALERTA_SUP20%_25,ALERTA_SUP20%_26,ALERTA_SUP20%_27
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,756405.0,774658.0,...,801658.0,233549217.0,239184885.0,2.475216e+08,0.323874,0.323874,0.323874,NO,NO,NO
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,232792812.0,238410227.0,...,246719991.0,233549217.0,239184885.0,2.475216e+08,99.676126,99.676126,99.676126,NO,NO,NO
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,20255.0,21570.0,...,23523.0,184753667.0,195776053.0,2.121398e+08,0.010963,0.011018,0.011088,NO,NO,NO
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,169733412.0,180754483.0,...,212116289.0,184753667.0,195776053.0,2.121398e+08,99.989037,99.988982,99.988912,NO,NO,NO
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,169831.0,176296.0,...,185880.0,97763757.0,101558146.0,1.070602e+08,0.173716,0.173591,0.173622,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,457. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PIURA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,120000000.0,120000000.0,...,804565289.0,956432060.0,977907989.0,1.009694e+09,79.687137,79.685859,79.684067,NO,NO,NO
70,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,458. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PUNO,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,7330000.0,7330000.0,...,605677648.0,573892246.0,587948106.0,6.084440e+08,99.545903,99.545763,99.545343,NO,NO,NO
71,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,459. GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,3000000.0,3500000.0,...,428449685.0,416930762.0,428804684.0,4.461417e+08,96.027360,96.031228,96.034443,NO,NO,NO
72,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,38931353.0,30035741.0,...,395360333.0,375108124.0,376141977.0,3.959571e+08,99.851767,99.847807,99.849281,NO,NO,NO


In [236]:
seg23 = pd.merge(bd_seg23, dict23, on=['COD_CADENA_GASTO'], how='left')
postmerge(seg23)
seg23 = pd.merge(seg23, dict231, on=['COD_ID'], how='left')
postmerge(seg23)
seg23.rename(columns={'PIA' : 'PIA_2023',
                      'PIM' : 'PIM_2023',
                      'TOTAL_DEVENGADO' : 'DEV_2023',
                      'CATEGORIA': 'CATEGORIA_GTO' }, inplace=True)
seg23= seg23[(seg23['COD_ID'].str[0] == '4') & (seg23['COD_ID'].str[3] == '-') ]
seg23= seg23[seg23['COD_FUENTE_RUBRO'] == 'FR.5.15']
seg23['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"
seg23['FUENTE'] =  "5. RECURSOS DETERMINADOS"
seg23['RUBRO'] =  "15. FONDO DE COMPENSACION REGIONAL - FONCOR"
seg23['TIPO_GEN_ING'] =  "2. OTRAS GENÉRICAS"
seg23 = seg23[['NIVEL_GOBIERNO', 'COD_ID', 'SECTOR', 'PLIEGO', 'COD_FUENTE_RUBRO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'COD_CADENA_GASTO', 'CATEGORIA_GTO', 'GENERICA', 'PIA_2023', 'PIM_2023', 'DEV_2023']].reset_index(drop=True)
seg23 = seg23.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA']).agg({'PIA_2023':'sum', 'PIM_2023':'sum', 'DEV_2023':'sum'}).reset_index()
seg23.info()
seg23.head(29575)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOBIERNO  59 non-null     object 
 1   SECTOR          59 non-null     object 
 2   PLIEGO          59 non-null     object 
 3   FUENTE          59 non-null     object 
 4   RUBRO           59 non-null     object 
 5   TIPO_GEN_ING    59 non-null     object 
 6   CATEGORIA_GTO   59 non-null     object 
 7   GENERICA        59 non-null     object 
 8   PIA_2023        59 non-null     int64  
 9   PIM_2023        59 non-null     int64  
 10  DEV_2023        59 non-null     float64
dtypes: float64(1), int64(2), object(8)
memory usage: 5.2+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PIA_2023,PIM_2023,DEV_2023
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0,1247447,1.242139e+06
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0,30631,0.000000e+00
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,182133390,195122507,1.916027e+08
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,119358216,119358216,6.510635e+07
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0,965992,2.912902e+05
5,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0,86531,7.653100e+04
6,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,92849541,92019065,8.467247e+07
7,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,2. PENSIONES Y OTRAS PRESTACIONES SOCIALES,0,1999290,0.000000e+00
8,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0,23791991,1.828716e+06
9,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0,11089420,1.038291e+07


In [237]:
seg24 = pd.merge(bd_seg24, dict24, on=['COD_CADENA_GASTO'], how='left')
postmerge(seg24)
seg24 = pd.merge(seg24, dict241, on=['COD_ID'], how='left')
postmerge(seg24)
seg24.rename(columns={'PIA' : 'PIA_2024',
                      'PIM' : 'PIM_2024',
                      'TOTAL_DEVENGADO' : 'DEV_2024',
                      'CATEGORIA': 'CATEGORIA_GTO' }, inplace=True)
seg24= seg24[(seg24['COD_ID'].str[0] == '4') & (seg24['COD_ID'].str[3] == '-') ]
seg24= seg24[seg24['COD_FUENTE_RUBRO'] == 'FR.5.15']
seg24['NIVEL_GOBIERNO'] =  "2. GOBIERNOS REGIONALES"
seg24['FUENTE'] =  "5. RECURSOS DETERMINADOS"
seg24['RUBRO'] =  "15. FONDO DE COMPENSACION REGIONAL - FONCOR"
seg24['TIPO_GEN_ING'] =  "2. OTRAS GENÉRICAS"
seg24 = seg24[['NIVEL_GOBIERNO', 'COD_ID', 'SECTOR', 'PLIEGO', 'COD_FUENTE_RUBRO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'COD_CADENA_GASTO', 'CATEGORIA_GTO', 'GENERICA', 'PIA_2024', 'PIM_2024', 'DEV_2024']].reset_index(drop=True)
seg24 = seg24.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA']).agg({'PIA_2024':'sum', 'PIM_2024':'sum', 'DEV_2024':'sum'}).reset_index()
seg24.info()
seg24.head(29575)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NIVEL_GOBIERNO  63 non-null     object 
 1   SECTOR          63 non-null     object 
 2   PLIEGO          63 non-null     object 
 3   FUENTE          63 non-null     object 
 4   RUBRO           63 non-null     object 
 5   TIPO_GEN_ING    63 non-null     object 
 6   CATEGORIA_GTO   63 non-null     object 
 7   GENERICA        63 non-null     object 
 8   PIA_2024        63 non-null     int64  
 9   PIM_2024        63 non-null     int64  
 10  DEV_2024        63 non-null     float64
dtypes: float64(1), int64(2), object(8)
memory usage: 5.5+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PIA_2024,PIM_2024,DEV_2024
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,4730349,5077303,0.00
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,199217049,200987049,35908351.07
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0,14113370,59642.95
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,208004354,219250456,27648121.75
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,6837569,6411193,12024.00
...,...,...,...,...,...,...,...,...,...,...,...
58,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,30200873,30200873,0.00
59,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0,1249444,826445.00
60,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,344179401,390388508,41769511.11
61,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,15495895,15495895,97370.73


In [238]:
Propuesta_RD_GGG1 = pd.merge(Propuesta_RD_GGG, seg23, on=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA'], how='left')
postmerge(Propuesta_RD_GGG1)
Propuesta_RD_GGG1 = pd.merge(Propuesta_RD_GGG1, seg24, on=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA'], how='left')
postmerge(Propuesta_RD_GGG1)
Propuesta_RD_GGG1=Propuesta_RD_GGG1[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA', 'PIA_2023', 'PIM_2023', 'DEV_2023','PIA_2024', 'PIM_2024', 'DEV_2024', 'PMG_2025','PMG_2026','PMG_2027', 
                                     'TOTAL_GEN_PLIEGO25', 'TOTAL_PLIEGO25', 'IND_GEN_PLIEGO25', 'ALERTA_SUP20%_25', 'TOTAL_GEN_PLIEGO26', 'TOTAL_PLIEGO26', 'IND_GEN_PLIEGO26', 'ALERTA_SUP20%_26', 'TOTAL_GEN_PLIEGO27', 'TOTAL_PLIEGO27', 'IND_GEN_PLIEGO27', 'ALERTA_SUP20%_27']]
Propuesta_RD_GGG1.info()
Propuesta_RD_GGG1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NIVEL_GOBIERNO      74 non-null     object 
 1   SECTOR              74 non-null     object 
 2   PLIEGO              74 non-null     object 
 3   FUENTE              74 non-null     object 
 4   RUBRO               74 non-null     object 
 5   TIPO_GEN_ING        74 non-null     object 
 6   CATEGORIA_GTO       74 non-null     object 
 7   GENERICA            74 non-null     object 
 8   PIA_2023            51 non-null     float64
 9   PIM_2023            51 non-null     float64
 10  DEV_2023            51 non-null     float64
 11  PIA_2024            53 non-null     float64
 12  PIM_2024            53 non-null     float64
 13  DEV_2024            53 non-null     float64
 14  PMG_2025            74 non-null     float64
 15  PMG_2026            74 non-null     float64
 16  PMG_2027  

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PIA_2023,PIM_2023,...,IND_GEN_PLIEGO25,ALERTA_SUP20%_25,TOTAL_GEN_PLIEGO26,TOTAL_PLIEGO26,IND_GEN_PLIEGO26,ALERTA_SUP20%_26,TOTAL_GEN_PLIEGO27,TOTAL_PLIEGO27,IND_GEN_PLIEGO27,ALERTA_SUP20%_27
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.0,1247447.0,...,0.323874,NO,774658.0,239184885.0,0.323874,NO,801658.0,247521649.0,0.323874,NO
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,182133390.0,195122507.0,...,99.676126,NO,238410227.0,239184885.0,99.676126,NO,246719991.0,247521649.0,99.676126,NO
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,NaN,NaN,...,0.010963,NO,21570.0,195776053.0,0.011018,NO,23523.0,212139812.0,0.011088,NO
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,119358216.0,119358216.0,...,99.989037,NO,195754483.0,195776053.0,99.988982,NO,212116289.0,212139812.0,99.988912,NO
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0.0,965992.0,...,0.173716,NO,176296.0,101558146.0,0.173591,NO,185880.0,107060198.0,0.173622,NO


In [239]:
Propuesta_RD_GGG2 = Propuesta_RD_GGG1.copy()
Propuesta_RD_GGG2 = Propuesta_RD_GGG2[Propuesta_RD_GGG2['GENERICA'] == "3. BIENES Y SERVICIOS" ]
outputFile = os.path.join(r'Y:\PMG 2025-2027\Outputs','Propuesta_RD_GG 2.3 Mantenimiento_PMG2025-2027_FONCOR_1'+ fecha_corte + ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    Propuesta_RD_GGG1.to_excel(ew, sheet_name="RD GASTO total", index = False)
    Propuesta_RD_GGG2.to_excel(ew, sheet_name="RD GG2.3", index = False)
#%%
        